## package installation and load packages


In [ ]:
### packages required
!pip install fair-esm
!pip install torch
!pip install tensorflow
!pip install sklearn
!pip install h5py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 2.9 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
import esm
import pandas as pd
import numpy as np
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv1D
from keras.layers import Dropout, AveragePooling1D, MaxPooling1D
from keras.models import Sequential,Model, load_model
from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint,LearningRateScheduler, EarlyStopping
import keras
from keras import backend as K
import tensorflow as tf
if tf.test.gpu_device_name():
    print('GPU found')
    tf.config.experimental.set_visible_devices(tf.config.list_physical_devices('GPU')[0], 'GPU') # set the deep learning with GPU
else:
    print("No GPU found")

GPU found


### peptide embeddings with differen pretrained model
https://github.com/facebookresearch/esm

Explaination of the memeory usage of the following models

sequence length > 900

2560 output dimension model might need 24 G GPU memory

5129 output dimension model, (in our attempts, 40 GB GPU memory is not enough)


In [ ]:
def esm_embeddings_320(esm2, esm2_alphabet, peptide_sequence_list):
  # NOTICE: ESM for embeddings is quite RAM usage, if your sequence is too long,
  #         or you have too many sequences for transformation in a single converting,
  #         you computer might automatically kill the job.
  import torch
  import esm
  import collections
  import pandas as pd
  import gc

  if torch.cuda.is_available():
    device = torch.device("cuda")
  else:
    device = torch.device("cpu")
  esm2 = esm2.eval().to(device)

  batch_converter = esm2_alphabet.get_batch_converter()

  # load the peptide sequence list into the bach_converter
  batch_labels, batch_strs, batch_tokens = batch_converter(peptide_sequence_list)
  batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)
  ## batch tokens are the embedding results of the whole data set

  batch_tokens = batch_tokens.to(device)

  # Extract per-residue representations (on CPU)
  with torch.no_grad():
      # Here we export the last layer of the EMS model output as the representation of the peptides
      # model'esm2_t6_8M_UR50D' only has 6 layers, and therefore repr_layers parameters is equal to 6
      results = esm2(batch_tokens, repr_layers=[6], return_contacts=False)
  token_representations = results["representations"][6].cpu()

  # Generate per-sequence representations via averaging
  # NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
  sequence_representations = []
  for i, tokens_len in enumerate(batch_lens):
      sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))
  # save dataset
  # sequence_representations is a list and each element is a tensor
  embeddings_results = collections.defaultdict(list)
  for i in range(len(sequence_representations)):
      # tensor can be transformed as numpy sequence_representations[0].numpy() or sequence_representations[0].to_list
      each_seq_rep = sequence_representations[i].tolist()
      for each_element in each_seq_rep:
          embeddings_results[i].append(each_element)
  embeddings_results = pd.DataFrame(embeddings_results).T
  del  batch_labels, batch_strs, batch_tokens, results, token_representations
  torch.cuda.empty_cache()
  gc.collect()
  return embeddings_results


In [ ]:
def esm_embeddings_480(esm2, esm2_alphabet, peptide_sequence_list):
  # NOTICE: ESM for embeddings is quite RAM usage, if your sequence is too long,
  #         or you have too many sequences for transformation in a single converting,
  #         you computer might automatically kill the job.
  import torch
  import esm
  import collections
  import pandas as pd
  import gc

  if torch.cuda.is_available():
    device = torch.device("cuda")
  else:
    device = torch.device("cpu")
  esm2 = esm2.eval().to(device)

  batch_converter = esm2_alphabet.get_batch_converter()

  # load the peptide sequence list into the bach_converter
  batch_labels, batch_strs, batch_tokens = batch_converter(peptide_sequence_list)
  batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)
  ## batch tokens are the embedding results of the whole data set

  batch_tokens = batch_tokens.to(device)

  # Extract per-residue representations (on CPU)
  with torch.no_grad():
      # Here we export the last layer of the EMS model output as the representation of the peptides
      # model'esm2_t12_35M_UR50D' only has 12 layers, and therefore repr_layers parameters is equal to 12
      results = esm2(batch_tokens, repr_layers=[12], return_contacts=False)
  token_representations = results["representations"][12].cpu()

  # Generate per-sequence representations via averaging
  # NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
  sequence_representations = []
  for i, tokens_len in enumerate(batch_lens):
      sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))
  # save dataset
  # sequence_representations is a list and each element is a tensor
  embeddings_results = collections.defaultdict(list)
  for i in range(len(sequence_representations)):
      # tensor can be transformed as numpy sequence_representations[0].numpy() or sequence_representations[0].to_list
      each_seq_rep = sequence_representations[i].tolist()
      for each_element in each_seq_rep:
          embeddings_results[i].append(each_element)
  embeddings_results = pd.DataFrame(embeddings_results).T
  del  batch_labels, batch_strs, batch_tokens, results, token_representations
  torch.cuda.empty_cache()
  gc.collect()
  return embeddings_results


In [ ]:
def esm_embeddings_640(esm2, esm2_alphabet, peptide_sequence_list):
  # NOTICE: ESM for embeddings is quite RAM usage, if your sequence is too long,
  #         or you have too many sequences for transformation in a single converting,
  #         you computer might automatically kill the job.
  import torch
  import esm
  import collections
  import pandas as pd
  import gc

  if torch.cuda.is_available():
    device = torch.device("cuda")
  else:
    device = torch.device("cpu")
  esm2 = esm2.eval().to(device)

  batch_converter = esm2_alphabet.get_batch_converter()

  # load the peptide sequence list into the bach_converter
  batch_labels, batch_strs, batch_tokens = batch_converter(peptide_sequence_list)
  batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)
  ## batch tokens are the embedding results of the whole data set

  batch_tokens = batch_tokens.to(device)

  # Extract per-residue representations (on CPU)
  with torch.no_grad():
      # Here we export the last layer of the EMS model output as the representation of the peptides
      # model'esm2_t30_150M_UR50D' only has 30 layers, and therefore repr_layers parameters is equal to 30
      results = esm2(batch_tokens, repr_layers=[30], return_contacts=False)
  token_representations = results["representations"][30].cpu()

  # Generate per-sequence representations via averaging
  # NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
  sequence_representations = []
  for i, tokens_len in enumerate(batch_lens):
      sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))
  # save dataset
  # sequence_representations is a list and each element is a tensor
  embeddings_results = collections.defaultdict(list)
  for i in range(len(sequence_representations)):
      # tensor can be transformed as numpy sequence_representations[0].numpy() or sequence_representations[0].to_list
      each_seq_rep = sequence_representations[i].tolist()
      for each_element in each_seq_rep:
          embeddings_results[i].append(each_element)
  embeddings_results = pd.DataFrame(embeddings_results).T
  del  batch_labels, batch_strs, batch_tokens, results, token_representations
  torch.cuda.empty_cache()
  gc.collect()
  return embeddings_results


In [ ]:
def esm_embeddings_1280(esm2, esm2_alphabet, peptide_sequence_list):
  # NOTICE: ESM for embeddings is quite RAM usage, if your sequence is too long,
  #         or you have too many sequences for transformation in a single converting,
  #         you computer might automatically kill the job.
  import torch
  import esm
  import collections
  import pandas as pd
  import gc

  if torch.cuda.is_available():
    device = torch.device("cuda")
  else:
    device = torch.device("cpu")
  esm2 = esm2.eval().to(device)

  batch_converter = esm2_alphabet.get_batch_converter()

  # load the peptide sequence list into the bach_converter
  batch_labels, batch_strs, batch_tokens = batch_converter(peptide_sequence_list)
  batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)
  ## batch tokens are the embedding results of the whole data set

  batch_tokens = batch_tokens.to(device)

  # Extract per-residue representations (on CPU)
  with torch.no_grad():
      # Here we export the last layer of the EMS model output as the representation of the peptides
      # model'esm2_t33_650M_UR50D' only has 33 layers, and therefore repr_layers parameters is equal to 33
      results = esm2(batch_tokens, repr_layers=[33], return_contacts=False)
  token_representations = results["representations"][33].cpu()

  # Generate per-sequence representations via averaging
  # NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
  sequence_representations = []
  for i, tokens_len in enumerate(batch_lens):
      sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))
  # save dataset
  # sequence_representations is a list and each element is a tensor
  embeddings_results = collections.defaultdict(list)
  for i in range(len(sequence_representations)):
      # tensor can be transformed as numpy sequence_representations[0].numpy() or sequence_representations[0].to_list
      each_seq_rep = sequence_representations[i].tolist()
      for each_element in each_seq_rep:
          embeddings_results[i].append(each_element)
  embeddings_results = pd.DataFrame(embeddings_results).T
  del  batch_labels, batch_strs, batch_tokens, results, token_representations
  torch.cuda.empty_cache()
  gc.collect()
  return embeddings_results


In [ ]:
def esm_embeddings_2560(esm2, esm2_alphabet, peptide_sequence_list):
  # NOTICE: ESM for embeddings is quite RAM usage, if your sequence is too long,
  #         or you have too many sequences for transformation in a single converting,
  #         you computer might automatically kill the job.
  import torch
  import esm
  import collections
  import pandas as pd
  import gc

  if torch.cuda.is_available():
    device = torch.device("cuda")
  else:
    device = torch.device("cpu")
  esm2 = esm2.eval().to(device)

  batch_converter = esm2_alphabet.get_batch_converter()

  # load the peptide sequence list into the bach_converter
  batch_labels, batch_strs, batch_tokens = batch_converter(peptide_sequence_list)
  batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)
  ## batch tokens are the embedding results of the whole data set

  batch_tokens = batch_tokens.to(device)

  # Extract per-residue representations (on CPU)
  with torch.no_grad():
      # Here we export the last layer of the EMS model output as the representation of the peptides
      # model'esm2_t36_3B_UR50D' only has 36 layers, and therefore repr_layers parameters is equal to 36
      results = esm2(batch_tokens, repr_layers=[36], return_contacts=False)
  token_representations = results["representations"][36].cpu()

  # Generate per-sequence representations via averaging
  # NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
  sequence_representations = []
  for i, tokens_len in enumerate(batch_lens):
      sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))
  # save dataset
  # sequence_representations is a list and each element is a tensor
  embeddings_results = collections.defaultdict(list)
  for i in range(len(sequence_representations)):
      # tensor can be transformed as numpy sequence_representations[0].numpy() or sequence_representations[0].to_list
      each_seq_rep = sequence_representations[i].tolist()
      for each_element in each_seq_rep:
          embeddings_results[i].append(each_element)
  embeddings_results = pd.DataFrame(embeddings_results).T
  del  batch_labels, batch_strs, batch_tokens, results, token_representations
  torch.cuda.empty_cache()
  gc.collect()
  return embeddings_results


In [ ]:
def esm_embeddings_5120(esm2, esm2_alphabet, peptide_sequence_list):
  # NOTICE: ESM for embeddings is quite RAM usage, if your sequence is too long,
  #         or you have too many sequences for transformation in a single converting,
  #         you computer might automatically kill the job.
  import torch
  import esm
  import collections
  import pandas as pd
  import gc

  if torch.cuda.is_available():
    device = torch.device("cuda")
  else:
    device = torch.device("cpu")
  esm2 = esm2.eval().to(device)

  batch_converter = esm2_alphabet.get_batch_converter()

  # load the peptide sequence list into the bach_converter
  batch_labels, batch_strs, batch_tokens = batch_converter(peptide_sequence_list)
  batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)
  ## batch tokens are the embedding results of the whole data set

  batch_tokens = batch_tokens.to(device)

  # Extract per-residue representations (on CPU)
  with torch.no_grad():
      # Here we export the last layer of the EMS model output as the representation of the peptides
      # model'esm2_t48_15B_UR50D' only has 48 layers, and therefore repr_layers parameters is equal to 48
      results = esm2(batch_tokens, repr_layers=[48], return_contacts=False)
  token_representations = results["representations"][48].cpu()

  # Generate per-sequence representations via averaging
  # NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
  sequence_representations = []
  for i, tokens_len in enumerate(batch_lens):
      sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))
  # save dataset
  # sequence_representations is a list and each element is a tensor
  embeddings_results = collections.defaultdict(list)
  for i in range(len(sequence_representations)):
      # tensor can be transformed as numpy sequence_representations[0].numpy() or sequence_representations[0].to_list
      each_seq_rep = sequence_representations[i].tolist()
      for each_element in each_seq_rep:
          embeddings_results[i].append(each_element)
  embeddings_results = pd.DataFrame(embeddings_results).T
  del  batch_labels, batch_strs, batch_tokens, results, token_representations
  torch.cuda.empty_cache()
  gc.collect()
  return embeddings_results


### connect with googledrive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls

Mounted at /content/drive
drive  sample_data


In [ ]:
import os
os.chdir('/content/drive/MyDrive/universal_allergenicity_new')

In [ ]:
!pwd
!ls

/content/drive/MyDrive/universal_allergenicity_new
 1280_dim.joblib
'1280_grid_performance output_repeat1.xlsx'
'1280_grid_performance output_repeat_2048.xlsx'
'1280_grid_performance output_repeat_256.xlsx'
'1280_grid_performance output_repeat_node_4096_2.xlsx'
'1280_grid_performance output_repeat_node_4096.xlsx'
'1280_grid_performance output_repeat_node_512_second.xlsx'
'1280_grid_performance output_repeat_node_8192_2.xlsx'
'1280_grid_performance output.xlsx'
'1280 performance output repeat 10.xlsx'
'1280 ROC.xlsx'
 1esm2_t6_8M_UR50D_unified_320_dimension.png
 2560_dim.joblib
'2560_grid_performance output_repeat_2.xlsx'
'2560_grid_performance output_repeat_node_4096_2.xlsx'
'2560_grid_performance output_repeat_node_512.xlsx'
'2560_grid_performance output.xlsx'
'2560 ROC.xlsx'
'2560 ten times repeat performance output_large_contained.xlsx'
'2560 ten times repeat performance output_small.xlsx'
 320_dim.joblib
'320_grid_performance output_repeat_2.xlsx'
'320_grid_performance output_repea

#### load packages

In [ ]:
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv1D
from keras.layers import Dropout, AveragePooling1D, MaxPooling1D
from keras.models import Sequential,Model, load_model
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint,LearningRateScheduler, EarlyStopping
import keras
from keras import backend as K
import tensorflow as tf
if tf.test.gpu_device_name():
    print('GPU found')
    tf.config.experimental.set_visible_devices(tf.config.list_physical_devices('GPU')[0], 'GPU') # set the deep learning with GPU
else:
    print("No GPU found")

GPU found


## Sequence embeddings with different pretrained protein language models

### 320 feature dimension embedding test

In [ ]:
import numpy as np
import pandas as pd
import esm
# select the ESM model for embeddings (you can select you desired model from https://github.com/facebookresearch/esm)
# NOTICE: if you choose other model, the following model architecture might not be very compitable
#         bseides,please revise the correspdoning parameters in esm_embeddings function (layers for feature extraction)
model, alphabet = esm.pretrained.esm2_t6_8M_UR50D()


# whole dataset loading and dataset splitting
dataset = pd.read_excel('allergens_dataset.xlsx',na_filter = False) # take care the NA sequence problem

# generate the peptide embeddings
sequence_list = dataset['sequence']
embeddings_results = pd.DataFrame()
for seq in sequence_list:
    # the setting is just following the input format setting in ESM model, [name,sequence]
    tuple_sequence = tuple([seq,seq])
    peptide_sequence_list = []
    peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information
    # employ ESM model for converting and save the converted data in csv format
    one_seq_embeddings = esm_embeddings_320(model, alphabet, peptide_sequence_list)
    embeddings_results= pd.concat([embeddings_results,one_seq_embeddings])
embeddings_results.to_csv('whole_sample_dataset_esm2_t6_8M_UR50D_unified_320_dimension.csv')

In [ ]:
import numpy as np
import pandas as pd
import esm
# select the ESM model for embeddings (you can select you desired model from https://github.com/facebookresearch/esm)
# NOTICE: if you choose other model, the following model architecture might not be very compitable
#         bseides,please revise the correspdoning parameters in esm_embeddings function (layers for feature extraction)
model, alphabet = esm.pretrained.esm2_t6_8M_UR50D()


# whole dataset loading and dataset splitting
dataset = pd.read_excel('allergens_dataset_pos1_neg0.xlsx',na_filter = False) # take care the NA sequence problem

# generate the peptide embeddings
sequence_list = dataset['sequence']
embeddings_results = pd.DataFrame()
for seq in sequence_list:
    # the setting is just following the input format setting in ESM model, [name,sequence]
    tuple_sequence = tuple([seq,seq])
    peptide_sequence_list = []
    peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information
    # employ ESM model for converting and save the converted data in csv format
    one_seq_embeddings = esm_embeddings_320(model, alphabet, peptide_sequence_list)
    embeddings_results= pd.concat([embeddings_results,one_seq_embeddings])
embeddings_results.to_csv('whole_sample_dataset_esm2_t6_8M_UR50D_unified_320_dimension_pos1_neg0.csv')

### 480 feature dimension embedding test

In [ ]:
import numpy as np
import pandas as pd
import esm
# select the ESM model for embeddings (you can select you desired model from https://github.com/facebookresearch/esm)
# NOTICE: if you choose other model, the following model architecture might not be very compitable
#         bseides,please revise the correspdoning parameters in esm_embeddings function (layers for feature extraction)
model, alphabet = esm.pretrained.esm2_t12_35M_UR50D()


# whole dataset loading and dataset splitting
dataset = pd.read_excel('allergens_dataset.xlsx',na_filter = False) # take care the NA sequence problem

# generate the peptide embeddings
sequence_list = dataset['sequence']
embeddings_results = pd.DataFrame()
for seq in sequence_list:
    # the setting is just following the input format setting in ESM model, [name,sequence]
    tuple_sequence = tuple([seq,seq])
    peptide_sequence_list = []
    peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information
    # employ ESM model for converting and save the converted data in csv format
    one_seq_embeddings = esm_embeddings_480(model, alphabet, peptide_sequence_list)
    embeddings_results= pd.concat([embeddings_results,one_seq_embeddings])
embeddings_results.to_csv('whole_sample_dataset_esm2_t12_35M_UR50D_unified_480_dimension.csv')

### 640 feature dimension embedding test

In [ ]:
import numpy as np
import pandas as pd
import esm
# select the ESM model for embeddings (you can select you desired model from https://github.com/facebookresearch/esm)
# NOTICE: if you choose other model, the following model architecture might not be very compitable
#         bseides,please revise the correspdoning parameters in esm_embeddings function (layers for feature extraction)
model, alphabet = esm.pretrained.esm2_t30_150M_UR50D()


# whole dataset loading and dataset splitting
dataset = pd.read_excel('allergens_dataset.xlsx',na_filter = False) # take care the NA sequence problem

# generate the peptide embeddings
sequence_list = dataset['sequence']
embeddings_results = pd.DataFrame()
for seq in sequence_list:
    # the setting is just following the input format setting in ESM model, [name,sequence]
    tuple_sequence = tuple([seq,seq])
    peptide_sequence_list = []
    peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information
    # employ ESM model for converting and save the converted data in csv format
    one_seq_embeddings = esm_embeddings_640(model, alphabet, peptide_sequence_list)
    embeddings_results= pd.concat([embeddings_results,one_seq_embeddings])
embeddings_results.to_csv('whole_sample_dataset_esm2_t30_150M_UR50D_unified_640_dimension.csv')

### 1280 feature dimension embedding test

In [ ]:
import numpy as np
import pandas as pd
import esm
# select the ESM model for embeddings (you can select you desired model from https://github.com/facebookresearch/esm)
# NOTICE: if you choose other model, the following model architecture might not be very compitable
#         bseides,please revise the correspdoning parameters in esm_embeddings function (layers for feature extraction)
model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()

# whole dataset loading and dataset splitting
dataset = pd.read_excel('allergens_dataset.xlsx',na_filter = False) # take care the NA sequence problem

# generate the peptide embeddings
sequence_list = dataset['sequence']
embeddings_results = pd.DataFrame()
for seq in sequence_list:
    # the setting is just following the input format setting in ESM model, [name,sequence]
    tuple_sequence = tuple([seq,seq])
    peptide_sequence_list = []
    peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information
    # employ ESM model for converting and save the converted data in csv format
    one_seq_embeddings = esm_embeddings_1280(model, alphabet, peptide_sequence_list)
    embeddings_results= pd.concat([embeddings_results,one_seq_embeddings])
embeddings_results.to_csv('whole_sample_dataset_esm2_t33_650M_UR50D_unified_1280_dimension.csv')

### 2560 feature dimension embedding test

In [ ]:
import numpy as np
import pandas as pd
import esm
# select the ESM model for embeddings (you can select you desired model from https://github.com/facebookresearch/esm)
# NOTICE: if you choose other model, the following model architecture might not be very compitable
#         bseides,please revise the correspdoning parameters in esm_embeddings function (layers for feature extraction)
model, alphabet = esm.pretrained.esm2_t36_3B_UR50D()


# whole dataset loading and dataset splitting
dataset = pd.read_excel('allergens_dataset.xlsx',na_filter = False) # take care the NA sequence problem
a=0
# generate the peptide embeddings
sequence_list = dataset['sequence']
embeddings_results = pd.DataFrame()
for seq in sequence_list:
    # the setting is just following the input format setting in ESM model, [name,sequence]
    tuple_sequence = tuple([seq,seq])
    peptide_sequence_list = []
    peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information
    # employ ESM model for converting and save the converted data in csv format
    one_seq_embeddings = esm_embeddings_2560(model, alphabet, peptide_sequence_list)
    embeddings_results= pd.concat([embeddings_results,one_seq_embeddings])
    a=a+1
    print(a)
embeddings_results.to_csv('whole_sample_dataset_esm2_t36_3B_UR50D_unified_2560_dimension.csv')

## grid search for hyperparameters of CNN model
Due to the enormous memory consumption and limitted dataset, we do not test 256 fileter size in CNN with 8192 units in dense layer.

### 320 feature dimension embedding test

In [ ]:
import numpy as np
import pandas as pd
import esm

# loading the y dataset for model development
dataset = pd.read_excel('allergens_dataset.xlsx',na_filter = False) # take care the NA sequence problem
y = dataset['label']
y = np.array(y) # transformed as np.array for CNN model

# read the peptide embeddings
X_data_name = 'whole_sample_dataset_esm2_t6_8M_UR50D_unified_320_dimension.csv'
X_data = pd.read_csv(X_data_name,header=0, index_col = 0,delimiter=',')
X = np.array(X_data)

# split dataset as training and test dataset as ratio of 8:2
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=123)

# normalize the X data range
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train) # normalize X to 0-1 range
X_test = scaler.transform(X_test)

ACC_collecton = []
CNN_channel = [16,32,64,128,256]
dense_node = [32,64,128,256,512,1024,2048,4096,8192]
kernel_size = [3,6,9,12]
stride_size = [1,2,4,8]
for i in range(len(CNN_channel)):
  for j in range(len(dense_node)):
    for k in range(len(kernel_size)):
      for l in range(len(stride_size)):
        inputShape=(320,1) # input feature size
        input = Input(inputShape)
        x = Conv1D(CNN_channel[i],(kernel_size[k]),strides = (stride_size[l]),name='layer_conv2',padding='same')(input)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = MaxPooling1D((2), name='MaxPool2',padding="same")(x)
        x = Dropout(0.15)(x)
        x = Flatten()(x)
        x = Dense(dense_node[j],activation = 'relu',name='fc1')(x)
        x = Dropout(0.15)(x)
        x = Dense(2,activation = 'softmax',name='fc2')(x)
        model = Model(inputs = input,outputs = x,name='Predict')
        # define SGD optimizer
        momentum = 0.5
        sgd = SGD(lr=0.01, momentum=momentum, decay=0.0, nesterov=False)
        # compile the model
        model.compile(loss='sparse_categorical_crossentropy',optimizer=sgd, metrics=['accuracy'])
        # learning deccay setting
        import math
        def step_decay(epoch): # gradually decrease the learning rate
            initial_lrate=0.1
            drop=0.6
            epochs_drop = 3.0
            lrate= initial_lrate * math.pow(drop,    # math.pow base raised to a power
                  math.floor((1+epoch)/epochs_drop)) # math.floor Round numbers down to the nearest integer
            return lrate
        lr = LearningRateScheduler(step_decay)
        # early stop setting
        early_stop = EarlyStopping(monitor='val_accuracy', patience = 20,verbose=0,restore_best_weights = True)
        # set checkpoint and save the best model
        mc = ModelCheckpoint('best_model_grid_320.h5',  monitor='val_accuracy', mode='max', verbose=0, save_best_only=True, save_weights_only=False)
        # summary the callbacks_list
        callbacks_list = [ lr , early_stop, mc]
        model_history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=200,callbacks=callbacks_list,batch_size = 32, verbose=0)
        # load the save best model
        saved_model = load_model('best_model_grid_320.h5')
        # result collection list
        # confusion matrix
        predicted_class= []
        predicted_protability = saved_model.predict(X_test,batch_size=1)
        for p in range(predicted_protability.shape[0]):
          index = np.where(predicted_protability[p] == np.amax(predicted_protability[p]))[0][0]
          predicted_class.append(index)
        predicted_class = np.array(predicted_class)
        y_true = y_test
        from sklearn.metrics import confusion_matrix
        import math
        # np.ravel() return a flatten 1D array
        TP, FP, FN, TN = confusion_matrix(y_true, predicted_class).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
        ACC = (TP+TN)/(TP+TN+FP+FN)
        ACC_collecton.append(ACC)
        print(ACC)
        del model
        import torch
        import gc
        torch.cuda.memory_reserved()
        gc.collect()

import collections
model_parameters =collections.defaultdict(list)
a=0
for i in range(len(CNN_channel)):
  for j in range(len(dense_node)):
    for k in range(len(kernel_size)):
      for l in range(len(stride_size)):
        iter_num = str(a)
        model_parameters[iter_num].append(CNN_channel[i])
        model_parameters[iter_num].append(dense_node[j])
        model_parameters[iter_num].append(kernel_size[k])
        model_parameters[iter_num].append(stride_size[l])
        model_parameters[iter_num].append(ACC_collecton[a])
        a=a+1
# export the DataFrame to an Excel file
pd.DataFrame(model_parameters).to_excel('320_grid_performance output.xlsx', index=False)

### 480 feature dimension embedding test

In [ ]:
import numpy as np
import pandas as pd
import esm

# loading the y dataset for model development
dataset = pd.read_excel('allergens_dataset.xlsx',na_filter = False) # take care the NA sequence problem
y = dataset['label']
y = np.array(y) # transformed as np.array for CNN model

# read the peptide embeddings
X_data_name = 'whole_sample_dataset_esm2_t12_35M_UR50D_unified_480_dimension.csv'
X_data = pd.read_csv(X_data_name,header=0, index_col = 0,delimiter=',')
X = np.array(X_data)

# split dataset as training and test dataset as ratio of 8:2
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=123)

# normalize the X data range
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train) # normalize X to 0-1 range
X_test = scaler.transform(X_test)

ACC_collecton = []
CNN_channel = [16,32,64,128,256]
dense_node = [32,64,128,256,512,1024,2048,4096,8192]
kernel_size = [3,6,9,12]
stride_size = [1,2,4,8]
for i in range(len(CNN_channel)):
  for j in range(len(dense_node)):
    for k in range(len(kernel_size)):
      for l in range(len(stride_size)):
        inputShape=(480,1) # input feature size
        input = Input(inputShape)
        x = Conv1D(CNN_channel[i],(kernel_size[k]),strides = (stride_size[l]),name='layer_conv2',padding='same')(input)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = MaxPooling1D((2), name='MaxPool2',padding="same")(x)
        x = Dropout(0.15)(x)
        x = Flatten()(x)
        x = Dense(dense_node[j],activation = 'relu',name='fc1')(x)
        x = Dropout(0.15)(x)
        x = Dense(2,activation = 'softmax',name='fc2')(x)
        model = Model(inputs = input,outputs = x,name='Predict')
        # define SGD optimizer
        momentum = 0.5
        sgd = SGD(lr=0.01, momentum=momentum, decay=0.0, nesterov=False)
        # compile the model
        model.compile(loss='sparse_categorical_crossentropy',optimizer=sgd, metrics=['accuracy'])
        # learning deccay setting
        import math
        def step_decay(epoch): # gradually decrease the learning rate
            initial_lrate=0.1
            drop=0.6
            epochs_drop = 3.0
            lrate= initial_lrate * math.pow(drop,    # math.pow base raised to a power
                  math.floor((1+epoch)/epochs_drop)) # math.floor Round numbers down to the nearest integer
            return lrate
        lr = LearningRateScheduler(step_decay)
        # early stop setting
        early_stop = EarlyStopping(monitor='val_accuracy', patience = 20,verbose=0,restore_best_weights = True)
        # set checkpoint and save the best model
        mc = ModelCheckpoint('best_model_grid_480.h5',  monitor='val_accuracy', mode='max', verbose=0, save_best_only=True, save_weights_only=False)
        # summary the callbacks_list
        callbacks_list = [ lr , early_stop, mc]
        model_history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=200,callbacks=callbacks_list,batch_size = 32, verbose=0)
        # load the save best model
        saved_model = load_model('best_model_grid_480.h5')
        # result collection list
        # confusion matrix
        predicted_class= []
        predicted_protability = saved_model.predict(X_test,batch_size=1)
        for p in range(predicted_protability.shape[0]):
          index = np.where(predicted_protability[p] == np.amax(predicted_protability[p]))[0][0]
          predicted_class.append(index)
        predicted_class = np.array(predicted_class)
        y_true = y_test
        from sklearn.metrics import confusion_matrix
        import math
        # np.ravel() return a flatten 1D array
        TP, FP, FN, TN = confusion_matrix(y_true, predicted_class).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
        ACC = (TP+TN)/(TP+TN+FP+FN)
        ACC_collecton.append(ACC)
        print(ACC)
        del model
        import torch
        import gc
        torch.cuda.memory_reserved()
        gc.collect()

import collections
model_parameters =collections.defaultdict(list)
a=0
for i in range(len(CNN_channel)):
  for j in range(len(dense_node)):
    for k in range(len(kernel_size)):
      for l in range(len(stride_size)):
        iter_num = str(a)
        model_parameters[iter_num].append(CNN_channel[i])
        model_parameters[iter_num].append(dense_node[j])
        model_parameters[iter_num].append(kernel_size[k])
        model_parameters[iter_num].append(stride_size[l])
        model_parameters[iter_num].append(ACC_collecton[a])
        a=a+1
# export the DataFrame to an Excel file
pd.DataFrame(model_parameters).to_excel('480_grid_performance output.xlsx', index=False)

### 640 feature dimension embedding test

In [ ]:
import numpy as np
import pandas as pd
import esm

# loading the y dataset for model development
dataset = pd.read_excel('allergens_dataset.xlsx',na_filter = False) # take care the NA sequence problem
y = dataset['label']
y = np.array(y) # transformed as np.array for CNN model

# read the peptide embeddings
X_data_name = 'whole_sample_dataset_esm2_t30_150M_UR50D_unified_640_dimension.csv'
X_data = pd.read_csv(X_data_name,header=0, index_col = 0,delimiter=',')
X = np.array(X_data)

# split dataset as training and test dataset as ratio of 8:2
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=123)

# normalize the X data range
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train) # normalize X to 0-1 range
X_test = scaler.transform(X_test)

ACC_collecton = []
CNN_channel = [16,32,64,128,256]
dense_node = [32,64,128,256,512,1024,2048,4096,8192]
kernel_size = [3,6,9,12]
stride_size = [1,2,4,8]
for i in range(len(CNN_channel)):
  for j in range(len(dense_node)):
    for k in range(len(kernel_size)):
      for l in range(len(stride_size)):
        inputShape=(640,1) # input feature size
        input = Input(inputShape)
        x = Conv1D(CNN_channel[i],(kernel_size[k]),strides = (stride_size[l]),name='layer_conv2',padding='same')(input)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = MaxPooling1D((2), name='MaxPool2',padding="same")(x)
        x = Dropout(0.15)(x)
        x = Flatten()(x)
        x = Dense(dense_node[j],activation = 'relu',name='fc1')(x)
        x = Dropout(0.15)(x)
        x = Dense(2,activation = 'softmax',name='fc2')(x)
        model = Model(inputs = input,outputs = x,name='Predict')
        # define SGD optimizer
        momentum = 0.5
        sgd = SGD(lr=0.01, momentum=momentum, decay=0.0, nesterov=False)
        # compile the model
        model.compile(loss='sparse_categorical_crossentropy',optimizer=sgd, metrics=['accuracy'])
        # learning deccay setting
        import math
        def step_decay(epoch): # gradually decrease the learning rate
            initial_lrate=0.1
            drop=0.6
            epochs_drop = 3.0
            lrate= initial_lrate * math.pow(drop,    # math.pow base raised to a power
                  math.floor((1+epoch)/epochs_drop)) # math.floor Round numbers down to the nearest integer
            return lrate
        lr = LearningRateScheduler(step_decay)
        # early stop setting
        early_stop = EarlyStopping(monitor='val_accuracy', patience = 20,verbose=0,restore_best_weights = True)
        # set checkpoint and save the best model
        mc = ModelCheckpoint('best_model_grid_640.h5',  monitor='val_accuracy', mode='max', verbose=0, save_best_only=True, save_weights_only=False)
        # summary the callbacks_list
        callbacks_list = [ lr , early_stop, mc]
        model_history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=200,callbacks=callbacks_list,batch_size = 32, verbose=0)
        # load the save best model
        saved_model = load_model('best_model_grid_640.h5')
        # result collection list
        # confusion matrix
        predicted_class= []
        predicted_protability = saved_model.predict(X_test,batch_size=1)
        for p in range(predicted_protability.shape[0]):
          index = np.where(predicted_protability[p] == np.amax(predicted_protability[p]))[0][0]
          predicted_class.append(index)
        predicted_class = np.array(predicted_class)
        y_true = y_test
        from sklearn.metrics import confusion_matrix
        import math
        # np.ravel() return a flatten 1D array
        TP, FP, FN, TN = confusion_matrix(y_true, predicted_class).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
        ACC = (TP+TN)/(TP+TN+FP+FN)
        ACC_collecton.append(ACC)
        print(ACC)
        del model
        import torch
        import gc
        torch.cuda.memory_reserved()
        gc.collect()
import collections
model_parameters =collections.defaultdict(list)
a=0
for i in range(len(CNN_channel)):
  for j in range(len(dense_node)):
    for k in range(len(kernel_size)):
      for l in range(len(stride_size)):
        iter_num = str(a)
        model_parameters[iter_num].append(CNN_channel[i])
        model_parameters[iter_num].append(dense_node[j])
        model_parameters[iter_num].append(kernel_size[k])
        model_parameters[iter_num].append(stride_size[l])
        model_parameters[iter_num].append(ACC_collecton[a])
        a=a+1
# export the DataFrame to an Excel file
pd.DataFrame(model_parameters).to_excel('640_grid_performance output.xlsx', index=False)

### 1280 feature dimension embedding test


In [ ]:
import numpy as np
import pandas as pd
import esm

# loading the y dataset for model development
dataset = pd.read_excel('allergens_dataset.xlsx',na_filter = False) # take care the NA sequence problem
y = dataset['label']
y = np.array(y) # transformed as np.array for CNN model

# read the peptide embeddings
X_data_name = 'whole_sample_dataset_esm2_t33_650M_UR50D_unified_1280_dimension.csv'
X_data = pd.read_csv(X_data_name,header=0, index_col = 0,delimiter=',')
X = np.array(X_data)

# split dataset as training and test dataset as ratio of 8:2
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=123)

# normalize the X data range
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train) # normalize X to 0-1 range
X_test = scaler.transform(X_test)

ACC_collecton = []
CNN_channel = [16,32,64,128,256]
dense_node = [32,64,128,256,512,1024,2048,4096,8192]
kernel_size = [3,6,9,12]
stride_size = [1,2,4,8]
for i in range(len(CNN_channel)):
  for j in range(len(dense_node)):
    for k in range(len(kernel_size)):
      for l in range(len(stride_size)):
        inputShape=(1280,1) # input feature size
        input = Input(inputShape)
        x = Conv1D(CNN_channel[i],(kernel_size[k]),strides = (stride_size[l]),name='layer_conv2',padding='same')(input)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = MaxPooling1D((2), name='MaxPool2',padding="same")(x)
        x = Dropout(0.15)(x)
        x = Flatten()(x)
        x = Dense(dense_node[j],activation = 'relu',name='fc1')(x)
        x = Dropout(0.15)(x)
        x = Dense(2,activation = 'softmax',name='fc2')(x)
        model = Model(inputs = input,outputs = x,name='Predict')
        # define SGD optimizer
        momentum = 0.5
        sgd = SGD(lr=0.01, momentum=momentum, decay=0.0, nesterov=False)
        # compile the model
        model.compile(loss='sparse_categorical_crossentropy',optimizer=sgd, metrics=['accuracy'])
        # learning deccay setting
        import math
        def step_decay(epoch): # gradually decrease the learning rate
            initial_lrate=0.1
            drop=0.6
            epochs_drop = 3.0
            lrate= initial_lrate * math.pow(drop,    # math.pow base raised to a power
                  math.floor((1+epoch)/epochs_drop)) # math.floor Round numbers down to the nearest integer
            return lrate
        lr = LearningRateScheduler(step_decay)
        # early stop setting
        early_stop = EarlyStopping(monitor='val_accuracy', patience = 20,verbose=0,restore_best_weights = True)
        # set checkpoint and save the best model
        mc = ModelCheckpoint('best_model_grid_1280.h5',  monitor='val_accuracy', mode='max', verbose=0, save_best_only=True, save_weights_only=False)
        # summary the callbacks_list
        callbacks_list = [ lr , early_stop, mc]
        model_history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=200,callbacks=callbacks_list,batch_size = 32, verbose=0)
        # load the save best model
        saved_model = load_model('best_model_grid_1280.h5')
        # result collection list
        # confusion matrix
        predicted_class= []
        predicted_protability = saved_model.predict(X_test,batch_size=1)
        for p in range(predicted_protability.shape[0]):
          index = np.where(predicted_protability[p] == np.amax(predicted_protability[p]))[0][0]
          predicted_class.append(index)
        predicted_class = np.array(predicted_class)
        y_true = y_test
        from sklearn.metrics import confusion_matrix
        import math
        # np.ravel() return a flatten 1D array
        TP, FP, FN, TN = confusion_matrix(y_true, predicted_class).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
        ACC = (TP+TN)/(TP+TN+FP+FN)
        ACC_collecton.append(ACC)
        print(ACC)
        del model
        import torch
        import gc
        torch.cuda.memory_reserved()
        gc.collect()
import collections
model_parameters =collections.defaultdict(list)
a=0
for i in range(len(CNN_channel)):
  for j in range(len(dense_node)):
    for k in range(len(kernel_size)):
      for l in range(len(stride_size)):
        iter_num = str(a)
        model_parameters[iter_num].append(CNN_channel[i])
        model_parameters[iter_num].append(dense_node[j])
        model_parameters[iter_num].append(kernel_size[k])
        model_parameters[iter_num].append(stride_size[l])
        model_parameters[iter_num].append(ACC_collecton[a])
        a=a+1
# export the DataFrame to an Excel file
pd.DataFrame(model_parameters).to_excel('1280_grid_performance output.xlsx', index=False)

### 2560 feature dimension embedding test
Due to the enourmous computational needs and also the limitted performance improivement, we do not test the 128 and 256 fileter size with 8196 units.

In [ ]:
import numpy as np
import pandas as pd
import esm

# loading the y dataset for model development
dataset = pd.read_excel('allergens_dataset.xlsx',na_filter = False) # take care the NA sequence problem
y = dataset['label']
y = np.array(y) # transformed as np.array for CNN model

# read the peptide embeddings
X_data_name = 'whole_sample_dataset_esm2_t36_3B_UR50D_unified_2560_dimension.csv'
X_data = pd.read_csv(X_data_name,header=0, index_col = 0,delimiter=',')
X = np.array(X_data)

# split dataset as training and test dataset as ratio of 8:2
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=123)

# normalize the X data range
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train) # normalize X to 0-1 range
X_test = scaler.transform(X_test)

ACC_collecton = []
CNN_channel = [16,32,64,128,256] #
dense_node = [32,64,128,256,512,1024,2048,4096]
kernel_size = [3,6,9,12]
stride_size = [1,2,4,8]
for i in range(len(CNN_channel)):
  for j in range(len(dense_node)):
    for k in range(len(kernel_size)):
      for l in range(len(stride_size)):
        inputShape=(2560,1) # input feature size
        input = Input(inputShape)
        x = Conv1D(CNN_channel[i],(kernel_size[k]),strides = (stride_size[l]),name='layer_conv2',padding='same')(input)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = MaxPooling1D((2), name='MaxPool2',padding="same")(x)
        x = Dropout(0.15)(x)
        x = Flatten()(x)
        x = Dense(dense_node[j],activation = 'relu',name='fc1')(x)
        x = Dropout(0.15)(x)
        x = Dense(2,activation = 'softmax',name='fc2')(x)
        model = Model(inputs = input,outputs = x,name='Predict')
        # define SGD optimizer
        momentum = 0.5
        sgd = SGD(lr=0.01, momentum=momentum, decay=0.0, nesterov=False)
        # compile the model
        model.compile(loss='sparse_categorical_crossentropy',optimizer=sgd, metrics=['accuracy'])
        # learning deccay setting
        import math
        def step_decay(epoch): # gradually decrease the learning rate
            initial_lrate=0.1
            drop=0.6
            epochs_drop = 3.0
            lrate= initial_lrate * math.pow(drop,    # math.pow base raised to a power
                  math.floor((1+epoch)/epochs_drop)) # math.floor Round numbers down to the nearest integer
            return lrate
        lr = LearningRateScheduler(step_decay)
        # early stop setting
        early_stop = EarlyStopping(monitor='val_accuracy', patience = 20,verbose=0,restore_best_weights = True)
        # set checkpoint and save the best model
        mc = ModelCheckpoint('best_model_grid_2560.h5',  monitor='val_accuracy', mode='max', verbose=0, save_best_only=True, save_weights_only=False)
        # summary the callbacks_list
        callbacks_list = [ lr , early_stop, mc]
        model_history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=200,callbacks=callbacks_list,batch_size = 32, verbose=0)
        # load the save best model
        saved_model = load_model('best_model_grid_2560.h5')
        # result collection list
        # confusion matrix
        predicted_class= []
        predicted_protability = saved_model.predict(X_test,batch_size=1)
        for p in range(predicted_protability.shape[0]):
          index = np.where(predicted_protability[p] == np.amax(predicted_protability[p]))[0][0]
          predicted_class.append(index)
        predicted_class = np.array(predicted_class)
        y_true = y_test
        from sklearn.metrics import confusion_matrix
        import math
        # np.ravel() return a flatten 1D array
        TP, FP, FN, TN = confusion_matrix(y_true, predicted_class).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
        ACC = (TP+TN)/(TP+TN+FP+FN)
        ACC_collecton.append(ACC)
        print(ACC)
        del model
        import torch
        import gc
        torch.cuda.memory_reserved()
        gc.collect()
import collections
model_parameters =collections.defaultdict(list)
a=0
for i in range(len(CNN_channel)):
  for j in range(len(dense_node)):
    for k in range(len(kernel_size)):
      for l in range(len(stride_size)):
        iter_num = str(a)
        model_parameters[iter_num].append(CNN_channel[i])
        model_parameters[iter_num].append(dense_node[j])
        model_parameters[iter_num].append(kernel_size[k])
        model_parameters[iter_num].append(stride_size[l])
        model_parameters[iter_num].append(ACC_collecton[a])
        a=a+1
# export the DataFrame to an Excel file
pd.DataFrame(model_parameters).to_excel('2560_grid_performance output.xlsx', index=False)

## 5-Fold cross validation and gather all the training dataset to generate the final model, evaluate the performance on test dataset

### 320 feature dimension embedding test

#### defined function for model development and evaluation

In [ ]:
def train_model(X_train, X_valid, y_train, y_valid, filter_size,kernel_size,stride_size,node_units):
  inputShape=(320,1) # input feature size
  input = Input(inputShape)
  x = Conv1D(filter_size,(kernel_size),strides = (stride_size),name='layer_conv2',padding='same')(input)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = MaxPooling1D((2), name='MaxPool2',padding="same")(x)
  x = Dropout(0.15)(x)
  x = Flatten()(x)
  x = Dense(node_units,activation = 'relu',name='fc1')(x)
  x = Dropout(0.15)(x)
  x = Dense(2,activation = 'softmax',name='fc2')(x)
  model = Model(inputs = input,outputs = x,name='Predict')

  # define SGD optimizer
  momentum = 0.5
  sgd = SGD(learning_rate=0.01, momentum=momentum, nesterov=False)
  # compile the model
  model.compile(loss='sparse_categorical_crossentropy',optimizer=sgd, metrics=['accuracy'])
  # learning deccay setting
  import math
  def step_decay(epoch): # gradually decrease the learning rate
      initial_lrate=0.1
      drop=0.6
      epochs_drop = 3.0
      lrate= initial_lrate * math.pow(drop,    # math.pow base raised to a power
            math.floor((1+epoch)/epochs_drop)) # math.floor Round numbers down to the nearest integer
      return lrate
  lr = LearningRateScheduler(step_decay)
  # early stop setting
  early_stop = EarlyStopping(monitor='val_accuracy', patience = 20,verbose=0,restore_best_weights = True)
  # set checkpoint and save the best model
  mc = ModelCheckpoint('best_model_grid_320.h5',  monitor='val_accuracy', mode='max', verbose=0, save_best_only=True, save_weights_only=False)
  # summary the callbacks_list
  callbacks_list = [ lr , early_stop, mc]
  model_history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=200,callbacks=callbacks_list,batch_size = 32, verbose=1)
  # load the save best model
  saved_model = load_model('best_model_grid_320.h5')
  return saved_model

  # import gc
  # del model
  # del saved_model
  # import torch
  # import gc
  # torch.cuda.memory_reserved()
  # gc.collect()

def check_performance(saved_model, X_test, y_test):
  # result collection list
  # confusion matrix
  predicted_class= []
  predicted_protability = saved_model.predict(X_test,batch_size=1)
  for p in range(predicted_protability.shape[0]):
    index = np.where(predicted_protability[p] == np.amax(predicted_protability[p]))[0][0]
    predicted_class.append(index)
  predicted_class = np.array(predicted_class)
  y_true = y_test
  from sklearn.metrics import confusion_matrix
  import math
  # np.ravel() return a flatten 1D array
  TN, FP, FN, TP = confusion_matrix(y_true, predicted_class).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
  ACC = (TP+TN)/(TP+TN+FP+FN)

  Sn = (TP/(TP+FN))
  Sp = (TN/(TN+FP))
  MCC = (TP*TN-FP*FN)/math.pow(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)),0.5)
  BACC = (0.5*TP/(TP+FN)+0.5*TN/(TN+FP))
  from sklearn.metrics import roc_auc_score
  AUC = roc_auc_score(y_test, predicted_protability[:,1])
  return ACC, Sn, Sp, MCC, BACC, AUC

#### 5 Fold CV for optimal hyperparrameter

In [ ]:
import os
os.chdir('/content/drive/MyDrive/universal_allergenicity_new')

In [ ]:
import numpy as np
import pandas as pd
import esm
import statistics
# whole dataset loading and dataset splitting
dataset = pd.read_excel('allergens_dataset.xlsx',na_filter = False) # take care the NA sequence problem

# generate the peptide embeddings
sequence_list = dataset['sequence']
y = dataset['label']
y = np.array(y) # transformed as np.array for CNN model

# read the peptide embeddings
X_data_name = 'whole_sample_dataset_esm2_t6_8M_UR50D_unified_320_dimension.csv'
X_data = pd.read_csv(X_data_name,header=0, index_col = 0,delimiter=',')
X = np.array(X_data)
from sklearn.model_selection import train_test_split
random_num = 1
X_train_whole, X_test, y_train_whole, y_test = train_test_split( X, y, test_size=0.2, random_state=random_num,shuffle=True, stratify = y)


In [ ]:
# set the selected hyperparameters from above grid search
CNN_channel = [32, 64, 16, 32, 32, 16, 16, 32, 32, 32, 128, 64, 32, 32, 64, 64, 128, 32, 64, 16, 128, 32, 16, 32, 32, 32]
dense_node = [32, 32, 64, 64, 128, 128, 256, 256, 512, 512, 8192, 4096, 8192, 4096, 4096, 4096, 4096, 4096, 8192, 1024, 2048, 2048, 2048, 1024, 512, 512]
kernel_size = [9, 6, 1, 6, 12, 12, 12, 3, 3, 9, 3, 12, 9, 6, 12, 12, 12, 12, 12, 9, 12, 12, 6, 6, 3, 12]
stride_size = [1, 4, 2, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 4, 4, 1, 2, 1, 8, 1, 1, 1, 1, 2]
print(len(CNN_channel),len(dense_node), len(kernel_size),len(stride_size))

26 26 26 26


In [ ]:
import os
os.chdir('/content')

In [ ]:
# 5-Fold CV
final_ACC_collection_cv = []
final_BACC_collection_cv = []
final_Sn_collection_cv = []
final_Sp_collection_cv = []
final_MCC_collection_cv = []
final_AUC_collection_cv = []

final_ACC_collection_test = []
final_BACC_collection_test = []
final_Sn_collection_test = []
final_Sp_collection_test = []
final_MCC_collection_test = []
final_AUC_collection_test = []

# split dataset
for i in range(len(CNN_channel)):
  # collect the value in cross validation
  ACC_collection_cv = []
  BACC_collection_cv = []
  Sn_collection_cv = []
  Sp_collection_cv = []
  MCC_collection_cv = []
  AUC_collection_cv = []

  ACC_collection_test = []
  BACC_collection_test = []
  Sn_collection_test = []
  Sp_collection_test = []
  MCC_collection_test = []
  AUC_collection_test = []
  from sklearn.model_selection import StratifiedKFold
  kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

  for train_ix, test_ix in kfold.split(X_train_whole, y_train_whole):
      X_train, X_valid = X[train_ix], X[test_ix]
      y_train, y_valid = y[train_ix], y[test_ix]
      X_train_only, X_train_indicator, y_train_only, y_train_indicator = train_test_split( X_train, y_train, test_size=0.25, random_state=random_num,shuffle=True, stratify = y_train)

      # train the model
      saved_model = train_model(X_train_only, X_train_indicator, y_train_only, y_train_indicator,CNN_channel[i],kernel_size[i],stride_size[i],dense_node[i])

      # ACC, Sn, Sp, MCC, BACC, AUC = check_performance(saved_model, X_valid, y_valid)

      # ACC_collection_cv.append(ACC)
      # BACC_collection_cv.append(BACC)
      # Sn_collection_cv.append(Sn)
      # Sp_collection_cv.append(Sp)
      # MCC_collection_cv.append(MCC)
      # AUC_collection_cv.append(AUC)

      ACC, Sn, Sp, MCC, BACC, AUC = check_performance(saved_model, X_valid, y_valid)
      print(ACC, Sn, Sp, MCC, BACC, AUC)
      ACC_collection_test.append(ACC)
      BACC_collection_test.append(BACC)
      Sn_collection_test.append(Sn)
      Sp_collection_test.append(Sp)
      MCC_collection_test.append(MCC)
      AUC_collection_test.append(AUC)

  # final_ACC_collection_cv.append(str(round(statistics.mean(ACC_collection_cv),3))+'±'+ str(round(statistics.stdev(ACC_collection_cv),3)))
  # final_BACC_collection_cv.append(str(round(statistics.mean(BACC_collection_cv),3))+'±'+str(round(statistics.stdev(BACC_collection_cv),3)))
  # final_Sn_collection_cv.append(str(round(statistics.mean(Sn_collection_cv),3))+'±'+str(round(statistics.stdev(Sn_collection_cv),3)))
  # final_Sp_collection_cv.append(str(round(statistics.mean(Sp_collection_cv),3))+'±'+str(round(statistics.stdev(Sp_collection_cv),3)))
  # final_MCC_collection_cv.append(str(round(statistics.mean(MCC_collection_cv),3))+'±'+str(round(statistics.stdev(MCC_collection_cv),3)))
  # final_AUC_collection_cv.append(str(round(statistics.mean(AUC_collection_cv),3))+'±'+str(round(statistics.stdev(AUC_collection_cv),3)))

  final_ACC_collection_test.append(str(round(statistics.mean(ACC_collection_test),3))+'±'+ str(round(statistics.stdev(ACC_collection_test),3)))
  final_BACC_collection_test.append(str(round(statistics.mean(BACC_collection_test),3))+'±'+str(round(statistics.stdev(BACC_collection_test),3)))
  final_Sn_collection_test.append(str(round(statistics.mean(Sn_collection_test),3))+'±'+str(round(statistics.stdev(Sn_collection_test),3)))
  final_Sp_collection_test.append(str(round(statistics.mean(Sp_collection_test),3))+'±'+str(round(statistics.stdev(Sp_collection_test),3)))
  final_MCC_collection_test.append(str(round(statistics.mean(MCC_collection_test),3))+'±'+str(round(statistics.stdev(MCC_collection_test),3)))
  final_AUC_collection_test.append(str(round(statistics.mean(AUC_collection_test),3))+'±'+str(round(statistics.stdev(AUC_collection_test),3)))


Epoch 1/200
241/241 [==============================] - 3s 5ms/step - loss: 0.8909 - accuracy: 0.6176 - val_loss: 0.6625 - val_accuracy: 0.6232 - lr: 0.1000
Epoch 2/200
 42/241 [====>.........................] - ETA: 0s - loss: 0.6765 - accuracy: 0.5960

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


241/241 [==============================] - 1s 5ms/step - loss: 0.6630 - accuracy: 0.6229 - val_loss: 0.6625 - val_accuracy: 0.6232 - lr: 0.1000
Epoch 3/200
241/241 [==============================] - 1s 5ms/step - loss: 0.6631 - accuracy: 0.6229 - val_loss: 0.6631 - val_accuracy: 0.6232 - lr: 0.0600
Epoch 4/200
241/241 [==============================] - 1s 5ms/step - loss: 0.6633 - accuracy: 0.6229 - val_loss: 0.6625 - val_accuracy: 0.6232 - lr: 0.0600
Epoch 5/200
241/241 [==============================] - 1s 4ms/step - loss: 0.6631 - accuracy: 0.6229 - val_loss: 0.6625 - val_accuracy: 0.6232 - lr: 0.0600
Epoch 6/200
241/241 [==============================] - 1s 4ms/step - loss: 0.6627 - accuracy: 0.6229 - val_loss: 0.6639 - val_accuracy: 0.6232 - lr: 0.0360
Epoch 7/200
241/241 [==============================] - 1s 4ms/step - loss: 0.6630 - accuracy: 0.6229 - val_loss: 0.6627 - val_accuracy: 0.6232 - lr: 0.0360
Epoch 8/200
241/241 [==============================] - 1s 5ms/step - loss: 0

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


241/241 [==============================] - 1s 4ms/step - loss: 0.3079 - accuracy: 0.8657 - val_loss: 0.2541 - val_accuracy: 0.8907 - lr: 0.1000
Epoch 3/200
241/241 [==============================] - 1s 4ms/step - loss: 0.2709 - accuracy: 0.8836 - val_loss: 0.2603 - val_accuracy: 0.8856 - lr: 0.0600
Epoch 4/200
241/241 [==============================] - 1s 5ms/step - loss: 0.2541 - accuracy: 0.8926 - val_loss: 0.2341 - val_accuracy: 0.8953 - lr: 0.0600
Epoch 5/200
241/241 [==============================] - 1s 4ms/step - loss: 0.2456 - accuracy: 0.8944 - val_loss: 0.2856 - val_accuracy: 0.8786 - lr: 0.0600
Epoch 6/200
241/241 [==============================] - 1s 5ms/step - loss: 0.2236 - accuracy: 0.9067 - val_loss: 0.2321 - val_accuracy: 0.8961 - lr: 0.0360
Epoch 7/200
241/241 [==============================] - 1s 5ms/step - loss: 0.2163 - accuracy: 0.9066 - val_loss: 0.2169 - val_accuracy: 0.9074 - lr: 0.0360
Epoch 8/200
241/241 [==============================] - 1s 4ms/step - loss: 0

...

241/241 [==============================] - 1s 5ms/step - loss: 0.2626 - accuracy: 0.8882 - val_loss: 0.2459 - val_accuracy: 0.8914 - lr: 0.1000
Epoch 3/200
241/241 [==============================] - 1s 5ms/step - loss: 0.2223 - accuracy: 0.9040 - val_loss: 0.2345 - val_accuracy: 0.8965 - lr: 0.0600
Epoch 4/200
241/241 [==============================] - 1s 5ms/step - loss: 0.2120 - accuracy: 0.9114 - val_loss: 0.2253 - val_accuracy: 0.9047 - lr: 0.0600
Epoch 5/200
241/241 [==============================] - 1s 5ms/step - loss: 0.2029 - accuracy: 0.9131 - val_loss: 0.2187 - val_accuracy: 0.9093 - lr: 0.0600
Epoch 6/200
241/241 [==============================] - 1s 4ms/step - loss: 0.1823 - accuracy: 0.9210 - val_loss: 0.2210 - val_accuracy: 0.9078 - lr: 0.0360
Epoch 7/200
241/241 [==============================] - 1s 5ms/step - loss: 0.1744 - accuracy: 0.9263 - val_loss: 0.2167 - val_accuracy: 0.9047 - lr: 0.0360
Epoch 8/200
241/241 [==============================] - 1s 5ms/step - loss: 0

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


241/241 [==============================] - 1s 5ms/step - loss: 0.2641 - accuracy: 0.8849 - val_loss: 0.2678 - val_accuracy: 0.8868 - lr: 0.1000
Epoch 3/200
241/241 [==============================] - 1s 5ms/step - loss: 0.2340 - accuracy: 0.8968 - val_loss: 0.2288 - val_accuracy: 0.9031 - lr: 0.0600
Epoch 4/200
241/241 [==============================] - 1s 4ms/step - loss: 0.2164 - accuracy: 0.9076 - val_loss: 0.2347 - val_accuracy: 0.8957 - lr: 0.0600
Epoch 5/200
241/241 [==============================] - 1s 5ms/step - loss: 0.2050 - accuracy: 0.9127 - val_loss: 0.2221 - val_accuracy: 0.9066 - lr: 0.0600
Epoch 6/200
241/241 [==============================] - 1s 5ms/step - loss: 0.1882 - accuracy: 0.9219 - val_loss: 0.2183 - val_accuracy: 0.9070 - lr: 0.0360
Epoch 7/200
241/241 [==============================] - 1s 5ms/step - loss: 0.1811 - accuracy: 0.9218 - val_loss: 0.2120 - val_accuracy: 0.9097 - lr: 0.0360
Epoch 8/200
241/241 [==============================] - 1s 4ms/step - loss: 0

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


241/241 [==============================] - 1s 5ms/step - loss: 0.2640 - accuracy: 0.8883 - val_loss: 0.2507 - val_accuracy: 0.8899 - lr: 0.1000
Epoch 3/200
241/241 [==============================] - 1s 5ms/step - loss: 0.2269 - accuracy: 0.9013 - val_loss: 0.2279 - val_accuracy: 0.9016 - lr: 0.0600
Epoch 4/200
241/241 [==============================] - 1s 5ms/step - loss: 0.2145 - accuracy: 0.9090 - val_loss: 0.2206 - val_accuracy: 0.9035 - lr: 0.0600
Epoch 5/200
241/241 [==============================] - 1s 5ms/step - loss: 0.2026 - accuracy: 0.9144 - val_loss: 0.2181 - val_accuracy: 0.9089 - lr: 0.0600
Epoch 6/200
241/241 [==============================] - 1s 4ms/step - loss: 0.1842 - accuracy: 0.9233 - val_loss: 0.2064 - val_accuracy: 0.9086 - lr: 0.0360
Epoch 7/200
241/241 [==============================] - 1s 5ms/step - loss: 0.1786 - accuracy: 0.9244 - val_loss: 0.2054 - val_accuracy: 0.9117 - lr: 0.0360
Epoch 8/200
241/241 [==============================] - 1s 4ms/step - loss: 0

In [ ]:
# performance in test
print(final_ACC_collection_test, '\n',final_BACC_collection_test, '\n',final_Sn_collection_test, '\n',final_Sp_collection_test, '\n',final_MCC_collection_test,'\n', final_AUC_collection_test)

['0.91±0.008', '0.909±0.006', '0.902±0.004', '0.912±0.01', '0.918±0.006', '0.918±0.005', '0.917±0.008', '0.914±0.015', '0.923±0.005', '0.92±0.005', '0.919±0.01', '0.921±0.007', '0.922±0.007', '0.924±0.009', '0.922±0.006', '0.923±0.008', '0.923±0.007', '0.923±0.007', '0.921±0.008', '0.917±0.008', '0.923±0.006', '0.92±0.004', '0.922±0.008', '0.922±0.008', '0.919±0.006', '0.921±0.008'] 
 ['0.903±0.013', '0.902±0.005', '0.893±0.006', '0.903±0.013', '0.912±0.008', '0.913±0.007', '0.911±0.01', '0.906±0.02', '0.918±0.007', '0.913±0.01', '0.913±0.012', '0.916±0.009', '0.915±0.008', '0.919±0.011', '0.916±0.007', '0.918±0.011', '0.918±0.009', '0.919±0.008', '0.917±0.009', '0.912±0.01', '0.918±0.008', '0.913±0.007', '0.919±0.01', '0.916±0.009', '0.912±0.009', '0.914±0.009'] 
 ['0.934±0.021', '0.93±0.014', '0.927±0.006', '0.938±0.008', '0.937±0.009', '0.933±0.006', '0.936±0.006', '0.939±0.005', '0.938±0.005', '0.94±0.01', '0.936±0.006', '0.937±0.008', '0.946±0.014', '0.941±0.008', '0.939±0.011', '

In [ ]:
# performance in test
print(final_ACC_collection_test, '\n',final_BACC_collection_test, '\n',final_Sn_collection_test, '\n',final_Sp_collection_test, '\n',final_MCC_collection_test,'\n', final_AUC_collection_test)


['0.906±0.014', '0.907±0.005', '0.906±0.005', '0.911±0.009', '0.917±0.01', '0.915±0.007', '0.918±0.007', '0.913±0.012', '0.92±0.006', '0.921±0.006', '0.911±0.022', '0.92±0.007', '0.925±0.007', '0.923±0.009', '0.92±0.009', '0.923±0.007', '0.925±0.008', '0.923±0.008', '0.923±0.007', '0.921±0.007', '0.921±0.009', '0.922±0.005', '0.918±0.009', '0.92±0.007', '0.918±0.008', '0.917±0.008'] 
 [0.9215420322878571, 0.9091034803906435, 0.8977857611101375, 0.9221628919838032, 0.9030276665975258] 
 [0.926859250153657, 0.9391086001255493, 0.934984520123839, 0.9342839429634222, 0.9386464263124604] 
 [0.9162248144220573, 0.8790983606557377, 0.860587002096436, 0.9100418410041841, 0.8674089068825911] 
 [0.8360943356038126, 0.8217553034712758, 0.8005996660692932, 0.8408594028885847, 0.8116126592175859] 
 [0.9774125784335259, 0.9713066515390079, 0.9686027870267603, 0.9795438215129688, 0.9735638541690672]


### final model

In [ ]:
# set the selected hyperparameters from above grid search
CNN_channel = [32,32,32,128,128]
dense_node = [512,8192,4096,4096,2048]
kernel_size = [3,9,6,12,12]
stride_size = [1,2,1,4,8]
print(len(CNN_channel),len(dense_node), len(kernel_size),len(stride_size))

5 5 5 5


In [ ]:
def train_model(X_train, y_train, filter_size,kernel_size,stride_size,node_units):
  inputShape=(320,1) # input feature size
  input = Input(inputShape)
  x = Conv1D(filter_size,(kernel_size),strides = (stride_size),name='layer_conv2',padding='same')(input)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = MaxPooling1D((2), name='MaxPool2',padding="same")(x)
  x = Dropout(0.15)(x)
  x = Flatten()(x)
  x = Dense(node_units,activation = 'relu',name='fc1')(x)
  x = Dropout(0.15)(x)
  x = Dense(2,activation = 'softmax',name='fc2')(x)
  model = Model(inputs = input,outputs = x,name='Predict')

  # define SGD optimizer
  momentum = 0.5
  sgd = SGD(learning_rate=0.01, momentum=momentum, nesterov=False)
  # compile the model
  model.compile(loss='sparse_categorical_crossentropy',optimizer=sgd, metrics=['accuracy'])
  # learning deccay setting
  import math
  def step_decay(epoch): # gradually decrease the learning rate
      initial_lrate=0.1
      drop=0.6
      epochs_drop = 3.0
      lrate= initial_lrate * math.pow(drop,    # math.pow base raised to a power
            math.floor((1+epoch)/epochs_drop)) # math.floor Round numbers down to the nearest integer
      return lrate
  lr = LearningRateScheduler(step_decay)
   # summary the callbacks_list
  callbacks_list = [ lr ]
  model_history = model.fit(X_train, y_train,  epochs=45,callbacks=callbacks_list,batch_size = 32, verbose=1)
  # load the save best model
  return model


def check_performance(saved_model, X_test, y_test):
  # result collection list
  # confusion matrix
  predicted_class= []
  predicted_protability = saved_model.predict(X_test,batch_size=1)
  for p in range(predicted_protability.shape[0]):
    index = np.where(predicted_protability[p] == np.amax(predicted_protability[p]))[0][0]
    predicted_class.append(index)
  predicted_class = np.array(predicted_class)
  y_true = y_test
  from sklearn.metrics import confusion_matrix
  import math
  # np.ravel() return a flatten 1D array
  TN, FP, FN, TP = confusion_matrix(y_true, predicted_class).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
  ACC = (TP+TN)/(TP+TN+FP+FN)

  Sn = (TP/(TP+FN))
  Sp = (TN/(TN+FP))
  MCC = (TP*TN-FP*FN)/math.pow(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)),0.5)
  BACC = (0.5*TP/(TP+FN)+0.5*TN/(TN+FP))
  from sklearn.metrics import roc_auc_score
  AUC = roc_auc_score(y_test, predicted_protability[:,1])
  return ACC, Sn, Sp, MCC, BACC, AUC

In [ ]:
import os
os.chdir('/content/drive/MyDrive/universal_allergenicity_new')

In [ ]:
import numpy as np
import pandas as pd
import esm
import statistics
# whole dataset loading and dataset splitting
dataset = pd.read_excel('allergens_dataset.xlsx',na_filter = False) # take care the NA sequence problem

# generate the peptide embeddings
sequence_list = dataset['sequence']
y = dataset['label']
y = np.array(y) # transformed as np.array for CNN model

# read the peptide embeddings
X_data_name = 'whole_sample_dataset_esm2_t6_8M_UR50D_unified_320_dimension.csv'
X_data = pd.read_csv(X_data_name,header=0, index_col = 0,delimiter=',')
X = np.array(X_data)
from sklearn.model_selection import train_test_split



In [ ]:
import os
os.chdir('/content')

In [ ]:
# final model
final_ACC_collection_test = []
final_BACC_collection_test = []
final_Sn_collection_test = []
final_Sp_collection_test = []
final_MCC_collection_test = []
final_AUC_collection_test = []
for i in range(len(CNN_channel)):
  # split dataset ten times and each times as 8:2 ratio for data division
  ACC_collection_test = []
  BACC_collection_test = []
  Sn_collection_test = []
  Sp_collection_test = []
  MCC_collection_test = []
  AUC_collection_test = []
  for a in range(10):
      random_num = a
      X_train_whole, X_test, y_train_whole, y_test = train_test_split( X, y, test_size=0.2, random_state=random_num,shuffle=True, stratify = y)

      # train the model with the optimal parameters in the cross validation
      saved_model = train_model(X_train_whole, y_train_whole,CNN_channel[i],kernel_size[i],stride_size[i],dense_node[i]) # optimal paraemters
      # performance evaluation at the independent test dataset
      ACC, Sn, Sp, MCC, BACC, AUC = check_performance(saved_model, X_test, y_test)
      print(ACC, Sn, Sp, MCC, BACC, AUC)
      ACC_collection_test.append(ACC)
      BACC_collection_test.append(BACC)
      Sn_collection_test.append(Sn)
      Sp_collection_test.append(Sp)
      MCC_collection_test.append(MCC)
      AUC_collection_test.append(AUC)

  import math
  final_ACC_collection_test.append(str(round(statistics.mean(ACC_collection_test),3))+'±'+ str(round(statistics.stdev(ACC_collection_test),3)))
  if any(math.isnan(i) for i in MCC_collection_test):
      final_BACC_collection_test.append('none')
      final_Sn_collection_test.append('none')
      final_Sp_collection_test.append('none')
      final_MCC_collection_test.append('none')
      final_AUC_collection_test.append('none')
  else:
      final_BACC_collection_test.append(str(round(statistics.mean(BACC_collection_test),3))+'±'+str(round(statistics.stdev(BACC_collection_test),3)))
      final_Sn_collection_test.append(str(round(statistics.mean(Sn_collection_test),3))+'±'+str(round(statistics.stdev(Sn_collection_test),3)))
      final_Sp_collection_test.append(str(round(statistics.mean(Sp_collection_test),3))+'±'+str(round(statistics.stdev(Sp_collection_test),3)))
      final_MCC_collection_test.append(str(round(statistics.mean(MCC_collection_test),3))+'±'+str(round(statistics.stdev(MCC_collection_test),3)))
      final_AUC_collection_test.append(str(round(statistics.mean(AUC_collection_test),3))+'±'+str(round(statistics.stdev(AUC_collection_test),3)))

# performance in test
print(final_ACC_collection_test, '\n',BACC_collection_test, '\n',Sn_collection_test, '\n',Sp_collection_test, '\n',MCC_collection_test,'\n', AUC_collection_test)


Epoch 1/45
402/402 [==============================] - 6s 4ms/step - loss: 0.6932 - accuracy: 0.8106 - lr: 0.1000
Epoch 2/45
402/402 [==============================] - 2s 4ms/step - loss: 0.2737 - accuracy: 0.8842 - lr: 0.1000
Epoch 3/45
402/402 [==============================] - 2s 4ms/step - loss: 0.2359 - accuracy: 0.8999 - lr: 0.0600
Epoch 4/45
402/402 [==============================] - 2s 4ms/step - loss: 0.2206 - accuracy: 0.9081 - lr: 0.0600
Epoch 5/45
402/402 [==============================] - 2s 4ms/step - loss: 0.2108 - accuracy: 0.9126 - lr: 0.0600
Epoch 6/45
402/402 [==============================] - 1s 4ms/step - loss: 0.1892 - accuracy: 0.9222 - lr: 0.0360
Epoch 7/45
402/402 [==============================] - 1s 4ms/step - loss: 0.1859 - accuracy: 0.9232 - lr: 0.0360
Epoch 8/45
402/402 [==============================] - 1s 4ms/step - loss: 0.1786 - accuracy: 0.9269 - lr: 0.0360
Epoch 9/45
402/402 [==============================] - 1s 4ms/step - loss: 0.1667 - accuracy: 0.9

### 480 feature dimension embedding test

#### defined function for model development and evaluation

In [ ]:
def train_model(X_train, X_valid, y_train, y_valid, filter_size,kernel_size,stride_size,node_units):
  inputShape=(480,1) # input feature size
  input = Input(inputShape)
  x = Conv1D(filter_size,(kernel_size),strides = (stride_size),name='layer_conv2',padding='same')(input)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = MaxPooling1D((2), name='MaxPool2',padding="same")(x)
  x = Dropout(0.15)(x)
  x = Flatten()(x)
  x = Dense(node_units,activation = 'relu',name='fc1')(x)
  x = Dropout(0.15)(x)
  x = Dense(2,activation = 'softmax',name='fc2')(x)
  model = Model(inputs = input,outputs = x,name='Predict')

  # define SGD optimizer
  momentum = 0.5
  sgd = SGD(learning_rate=0.01, momentum=momentum, nesterov=False)
  # compile the model
  model.compile(loss='sparse_categorical_crossentropy',optimizer=sgd, metrics=['accuracy'])
  # learning deccay setting
  import math
  def step_decay(epoch): # gradually decrease the learning rate
      initial_lrate=0.1
      drop=0.6
      epochs_drop = 3.0
      lrate= initial_lrate * math.pow(drop,    # math.pow base raised to a power
            math.floor((1+epoch)/epochs_drop)) # math.floor Round numbers down to the nearest integer
      return lrate
  lr = LearningRateScheduler(step_decay)
  # early stop setting
  early_stop = EarlyStopping(monitor='val_accuracy', patience = 20,verbose=0,restore_best_weights = True)
  # set checkpoint and save the best model
  mc = ModelCheckpoint('best_model_grid_480.h5',  monitor='val_accuracy', mode='max', verbose=0, save_best_only=True, save_weights_only=False)
  # summary the callbacks_list
  callbacks_list = [ lr , early_stop, mc]
  model_history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=200,callbacks=callbacks_list,batch_size = 32, verbose=1)
  # load the save best model
  saved_model = load_model('best_model_grid_480.h5')
  return saved_model

  # import gc
  # del model
  # del saved_model
  # import torch
  # import gc
  # torch.cuda.memory_reserved()
  # gc.collect()

def check_performance(saved_model, X_test, y_test):
  # result collection list
  # confusion matrix
  predicted_class= []
  predicted_protability = saved_model.predict(X_test,batch_size=1)
  for p in range(predicted_protability.shape[0]):
    index = np.where(predicted_protability[p] == np.amax(predicted_protability[p]))[0][0]
    predicted_class.append(index)
  predicted_class = np.array(predicted_class)
  y_true = y_test
  from sklearn.metrics import confusion_matrix
  import math
  # np.ravel() return a flatten 1D array
  TN, FP, FN, TP = confusion_matrix(y_true, predicted_class).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
  ACC = (TP+TN)/(TP+TN+FP+FN)

  Sn = (TP/(TP+FN))
  Sp = (TN/(TN+FP))
  MCC = (TP*TN-FP*FN)/math.pow(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)),0.5)
  BACC = (0.5*TP/(TP+FN)+0.5*TN/(TN+FP))
  from sklearn.metrics import roc_auc_score
  AUC = roc_auc_score(y_test, predicted_protability[:,1])
  return ACC, Sn, Sp, MCC, BACC, AUC

#### 5 Fold CV for optimal hyperparrameter

In [ ]:
import os
os.chdir('/content/drive/MyDrive/universal_allergenicity_new')

In [ ]:
import numpy as np
import pandas as pd
import esm
import statistics
# whole dataset loading and dataset splitting
dataset = pd.read_excel('allergens_dataset.xlsx',na_filter = False) # take care the NA sequence problem

# generate the peptide embeddings
sequence_list = dataset['sequence']
y = dataset['label']
y = np.array(y) # transformed as np.array for CNN model

# read the peptide embeddings
X_data_name = 'whole_sample_dataset_esm2_t12_35M_UR50D_unified_480_dimension.csv'
X_data = pd.read_csv(X_data_name,header=0, index_col = 0,delimiter=',')
X = np.array(X_data)
from sklearn.model_selection import train_test_split
random_num = 1
X_train_whole, X_test, y_train_whole, y_test = train_test_split( X, y, test_size=0.2, random_state=random_num,shuffle=True, stratify = y)


In [ ]:
# set the selected hyperparameters from above grid search
CNN_channel = [128, 32, 64, 64, 32, 16, 16, 16, 16, 16, 64, 64, 64, 16, 128, 16, 32, 32, 16, 32, 32, 64, 32, 32, 16, 16, 32, 32, 16, 32]
dense_node = [8192, 4096, 8192, 8192, 4096, 8192, 8192, 2048, 8192, 2048, 2048, 2048, 1024, 2048, 1024, 2048, 512, 1024, 512, 32, 32, 64, 64, 128, 128, 256, 256, 512, 512, 512]
kernel_size = [9, 6, 6, 9, 9, 12, 6, 12, 9, 9, 6, 12, 12, 6, 12, 9, 6, 12, 12, 6, 12, 6, 9, 9, 12, 6, 9, 6, 12, 6]
stride_size = [8, 1, 4, 8, 4, 2, 2, 2, 2, 2, 8, 4, 4, 1, 4, 1, 4, 1, 1, 1, 1, 2, 4, 1, 1, 2, 1, 4, 1, 2]
print(len(CNN_channel),len(dense_node), len(kernel_size),len(stride_size))

30 30 30 30


In [ ]:
import os
os.chdir('/content')

In [ ]:
# 5-Fold CV
final_ACC_collection_cv = []
final_BACC_collection_cv = []
final_Sn_collection_cv = []
final_Sp_collection_cv = []
final_MCC_collection_cv = []
final_AUC_collection_cv = []

final_ACC_collection_test = []
final_BACC_collection_test = []
final_Sn_collection_test = []
final_Sp_collection_test = []
final_MCC_collection_test = []
final_AUC_collection_test = []

# split dataset
# for i in range(len(CNN_channel)-15):
for a in range(12):
  i = a+20
  # collect the value in cross validation
  ACC_collection_cv = []
  BACC_collection_cv = []
  Sn_collection_cv = []
  Sp_collection_cv = []
  MCC_collection_cv = []
  AUC_collection_cv = []

  ACC_collection_test = []
  BACC_collection_test = []
  Sn_collection_test = []
  Sp_collection_test = []
  MCC_collection_test = []
  AUC_collection_test = []
  from sklearn.model_selection import StratifiedKFold
  kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

  for train_ix, test_ix in kfold.split(X_train_whole, y_train_whole):
      X_train, X_valid = X[train_ix], X[test_ix]
      y_train, y_valid = y[train_ix], y[test_ix]
      X_train_only, X_train_indicator, y_train_only, y_train_indicator = train_test_split( X_train, y_train, test_size=0.25, random_state=random_num,shuffle=True, stratify = y_train)

      # train the model
      saved_model = train_model(X_train_only, X_train_indicator, y_train_only, y_train_indicator,CNN_channel[i],kernel_size[i],stride_size[i],dense_node[i])

      ACC, Sn, Sp, MCC, BACC, AUC = check_performance(saved_model, X_valid, y_valid)
      print(ACC, Sn, Sp, MCC, BACC, AUC)
      ACC_collection_test.append(ACC)
      BACC_collection_test.append(BACC)
      Sn_collection_test.append(Sn)
      Sp_collection_test.append(Sp)
      MCC_collection_test.append(MCC)
      AUC_collection_test.append(AUC)

  final_ACC_collection_test.append(str(round(statistics.mean(ACC_collection_test),3))+'±'+ str(round(statistics.stdev(ACC_collection_test),3)))
  final_BACC_collection_test.append(str(round(statistics.mean(BACC_collection_test),3))+'±'+str(round(statistics.stdev(BACC_collection_test),3)))
  final_Sn_collection_test.append(str(round(statistics.mean(Sn_collection_test),3))+'±'+str(round(statistics.stdev(Sn_collection_test),3)))
  final_Sp_collection_test.append(str(round(statistics.mean(Sp_collection_test),3))+'±'+str(round(statistics.stdev(Sp_collection_test),3)))
  final_MCC_collection_test.append(str(round(statistics.mean(MCC_collection_test),3))+'±'+str(round(statistics.stdev(MCC_collection_test),3)))
  final_AUC_collection_test.append(str(round(statistics.mean(AUC_collection_test),3))+'±'+str(round(statistics.stdev(AUC_collection_test),3)))


Epoch 1/200
241/241 [==============================] - 2s 6ms/step - loss: 0.7116 - accuracy: 0.6238 - val_loss: 0.6630 - val_accuracy: 0.6232 - lr: 0.1000
Epoch 2/200
 40/241 [===>..........................] - ETA: 0s - loss: 0.6449 - accuracy: 0.6555

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


241/241 [==============================] - 1s 5ms/step - loss: 0.6630 - accuracy: 0.6229 - val_loss: 0.6625 - val_accuracy: 0.6232 - lr: 0.1000
Epoch 3/200
241/241 [==============================] - 1s 4ms/step - loss: 0.6632 - accuracy: 0.6229 - val_loss: 0.6625 - val_accuracy: 0.6232 - lr: 0.0600
Epoch 4/200
241/241 [==============================] - 1s 4ms/step - loss: 0.6631 - accuracy: 0.6229 - val_loss: 0.6626 - val_accuracy: 0.6232 - lr: 0.0600
Epoch 5/200
241/241 [==============================] - 1s 4ms/step - loss: 0.6633 - accuracy: 0.6229 - val_loss: 0.6625 - val_accuracy: 0.6232 - lr: 0.0600
Epoch 6/200
241/241 [==============================] - 1s 4ms/step - loss: 0.6629 - accuracy: 0.6229 - val_loss: 0.6632 - val_accuracy: 0.6232 - lr: 0.0360
Epoch 7/200
241/241 [==============================] - 1s 4ms/step - loss: 0.6631 - accuracy: 0.6229 - val_loss: 0.6625 - val_accuracy: 0.6232 - lr: 0.0360
Epoch 8/200
241/241 [==============================] - 1s 4ms/step - loss: 0

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


241/241 [==============================] - 1s 4ms/step - loss: 0.6344 - accuracy: 0.6572 - val_loss: 0.6613 - val_accuracy: 0.6261 - lr: 0.1000
Epoch 3/200
241/241 [==============================] - 1s 4ms/step - loss: 0.6615 - accuracy: 0.6263 - val_loss: 0.6618 - val_accuracy: 0.6261 - lr: 0.0600
Epoch 4/200
241/241 [==============================] - 1s 4ms/step - loss: 0.6614 - accuracy: 0.6263 - val_loss: 0.6614 - val_accuracy: 0.6261 - lr: 0.0600
Epoch 5/200
241/241 [==============================] - 1s 4ms/step - loss: 0.6614 - accuracy: 0.6263 - val_loss: 0.6611 - val_accuracy: 0.6261 - lr: 0.0600
Epoch 6/200
241/241 [==============================] - 1s 4ms/step - loss: 0.6488 - accuracy: 0.6263 - val_loss: 0.6699 - val_accuracy: 0.6261 - lr: 0.0360
Epoch 7/200
241/241 [==============================] - 1s 5ms/step - loss: 0.5168 - accuracy: 0.7070 - val_loss: 0.3806 - val_accuracy: 0.8572 - lr: 0.0360
Epoch 8/200
241/241 [==============================] - 1s 5ms/step - loss: 0

...

241/241 [==============================] - 2s 5ms/step - loss: 1.1916 - accuracy: 0.6254 - val_loss: 0.6604 - val_accuracy: 0.6272 - lr: 0.1000
Epoch 2/200
 43/241 [====>.........................] - ETA: 0s - loss: 0.6560 - accuracy: 0.6374

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


241/241 [==============================] - 1s 4ms/step - loss: 0.6612 - accuracy: 0.6274 - val_loss: 0.6604 - val_accuracy: 0.6272 - lr: 0.1000
Epoch 3/200
241/241 [==============================] - 1s 4ms/step - loss: 0.6614 - accuracy: 0.6272 - val_loss: 0.6608 - val_accuracy: 0.6272 - lr: 0.0600
Epoch 4/200
241/241 [==============================] - 1s 4ms/step - loss: 0.6607 - accuracy: 0.6275 - val_loss: 0.6610 - val_accuracy: 0.6272 - lr: 0.0600
Epoch 5/200
241/241 [==============================] - 1s 4ms/step - loss: 0.6608 - accuracy: 0.6275 - val_loss: 0.6604 - val_accuracy: 0.6272 - lr: 0.0600
Epoch 6/200
241/241 [==============================] - 1s 4ms/step - loss: 0.6605 - accuracy: 0.6275 - val_loss: 0.6608 - val_accuracy: 0.6272 - lr: 0.0360
Epoch 7/200
241/241 [==============================] - 1s 4ms/step - loss: 0.6607 - accuracy: 0.6275 - val_loss: 0.6604 - val_accuracy: 0.6272 - lr: 0.0360
Epoch 8/200
241/241 [==============================] - 1s 4ms/step - loss: 0

<ipython-input-13-412af7b487d6>:67: RuntimeWarning: invalid value encountered in divide
  MCC = (TP*TN-FP*FN)/math.pow(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)),0.5)


ValueError: ignored

In [ ]:
# performance in test
print(final_ACC_collection_test, '\n',final_BACC_collection_test, '\n',final_Sn_collection_test, '\n',final_Sp_collection_test, '\n',final_MCC_collection_test,'\n', final_AUC_collection_test)

['0.929±0.006', '0.921±0.006', '0.928±0.006', '0.928±0.005', '0.928±0.008', '0.924±0.008', '0.929±0.007', '0.922±0.012', '0.926±0.006', '0.925±0.006', '0.93±0.005', '0.928±0.007', '0.927±0.008', '0.923±0.004', '0.928±0.008', '0.919±0.013', '0.929±0.008', '0.906±0.033'] 
 ['0.926±0.009', '0.915±0.005', '0.923±0.009', '0.923±0.007', '0.925±0.011', '0.92±0.008', '0.925±0.008', '0.918±0.016', '0.922±0.006', '0.919±0.008', '0.927±0.006', '0.924±0.008', '0.922±0.009', '0.917±0.006', '0.923±0.01', '0.913±0.015', '0.925±0.01', '0.894±0.045'] 
 ['0.938±0.003', '0.94±0.012', '0.942±0.013', '0.945±0.01', '0.938±0.011', '0.938±0.011', '0.94±0.006', '0.937±0.004', '0.939±0.009', '0.942±0.005', '0.94±0.011', '0.941±0.005', '0.943±0.01', '0.943±0.009', '0.943±0.008', '0.937±0.012', '0.942±0.005', '0.942±0.006'] 
 ['0.915±0.017', '0.889±0.012', '0.904±0.026', '0.901±0.018', '0.912±0.025', '0.901±0.013', '0.91±0.011', '0.899±0.028', '0.906±0.01', '0.896±0.017', '0.913±0.016', '0.906±0.012', '0.901±0.01

In [ ]:
# there is an bad initialization of parameters
# since the i = 18; the rest hyperparameters are rerunned
# performance in test
print(final_ACC_collection_test, '\n',final_BACC_collection_test, '\n',final_Sn_collection_test, '\n',final_Sp_collection_test, '\n',final_MCC_collection_test,'\n', final_AUC_collection_test)

['0.919±0.007', '0.683±0.133'] 
 ['0.912±0.008', '0.582±0.184'] 
 ['0.94±0.006', '0.989±0.024'] 
 ['0.883±0.012', '0.175±0.392'] 
 ['0.826±0.015'] 
 ['0.976±0.003']


In [ ]:
# there is an bad initialization of parameters
# since the i = 18; the rest hyperparameters are rerunned
# performance in test
print(final_ACC_collection_test, '\n',final_BACC_collection_test, '\n',final_Sn_collection_test, '\n',final_Sp_collection_test, '\n',final_MCC_collection_test,'\n', final_AUC_collection_test)

NameError: ignored

### final model

In [ ]:
# set the selected hyperparameters from above grid search
CNN_channel = [128,64,64,32,16,64,128,32,32]
dense_node = [8192,8192,8192,4096,8192,2048,1024,512,512]
kernel_size = [9,6,9,9,6,12,12,6,6]
stride_size = [8,4,8,4,2,4,4,4,4]
print(len(CNN_channel),len(dense_node), len(kernel_size),len(stride_size))

5 5 5 5


In [ ]:
def train_model(X_train, y_train, filter_size,kernel_size,stride_size,node_units):
  inputShape=(480,1) # input feature size
  input = Input(inputShape)
  x = Conv1D(filter_size,(kernel_size),strides = (stride_size),name='layer_conv2',padding='same')(input)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = MaxPooling1D((2), name='MaxPool2',padding="same")(x)
  x = Dropout(0.15)(x)
  x = Flatten()(x)
  x = Dense(node_units,activation = 'relu',name='fc1')(x)
  x = Dropout(0.15)(x)
  x = Dense(2,activation = 'softmax',name='fc2')(x)
  model = Model(inputs = input,outputs = x,name='Predict')

  # define SGD optimizer
  momentum = 0.5
  sgd = SGD(learning_rate=0.01, momentum=momentum, nesterov=False)
  # compile the model
  model.compile(loss='sparse_categorical_crossentropy',optimizer=sgd, metrics=['accuracy'])
  # learning deccay setting
  import math
  def step_decay(epoch): # gradually decrease the learning rate
      initial_lrate=0.1
      drop=0.6
      epochs_drop = 3.0
      lrate= initial_lrate * math.pow(drop,    # math.pow base raised to a power
            math.floor((1+epoch)/epochs_drop)) # math.floor Round numbers down to the nearest integer
      return lrate
  lr = LearningRateScheduler(step_decay)
   # summary the callbacks_list
  callbacks_list = [ lr ]
  model_history = model.fit(X_train, y_train,  epochs=45,callbacks=callbacks_list,batch_size = 32, verbose=1)
  # load the save best model
  return model


def check_performance(saved_model, X_test, y_test):
  # result collection list
  # confusion matrix
  predicted_class= []
  predicted_protability = saved_model.predict(X_test,batch_size=1)
  for p in range(predicted_protability.shape[0]):
    index = np.where(predicted_protability[p] == np.amax(predicted_protability[p]))[0][0]
    predicted_class.append(index)
  predicted_class = np.array(predicted_class)
  y_true = y_test
  from sklearn.metrics import confusion_matrix
  import math
  # np.ravel() return a flatten 1D array
  TN, FP, FN, TP = confusion_matrix(y_true, predicted_class).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
  ACC = (TP+TN)/(TP+TN+FP+FN)

  Sn = (TP/(TP+FN))
  Sp = (TN/(TN+FP))
  MCC = (TP*TN-FP*FN)/math.pow(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)),0.5)
  BACC = (0.5*TP/(TP+FN)+0.5*TN/(TN+FP))
  from sklearn.metrics import roc_auc_score
  AUC = roc_auc_score(y_test, predicted_protability[:,1])
  return ACC, Sn, Sp, MCC, BACC, AUC

In [ ]:
import os
os.chdir('/content/drive/MyDrive/universal_allergenicity_new')

In [ ]:
import numpy as np
import pandas as pd
import esm
import statistics
# whole dataset loading and dataset splitting
dataset = pd.read_excel('allergens_dataset.xlsx',na_filter = False) # take care the NA sequence problem

# generate the peptide embeddings
sequence_list = dataset['sequence']
y = dataset['label']
y = np.array(y) # transformed as np.array for CNN model

# read the peptide embeddings
X_data_name = 'whole_sample_dataset_esm2_t12_35M_UR50D_unified_480_dimension.csv'
X_data = pd.read_csv(X_data_name,header=0, index_col = 0,delimiter=',')
X = np.array(X_data)
from sklearn.model_selection import train_test_split



In [ ]:
import os
os.chdir('/content')

In [ ]:
# final model
final_ACC_collection_test = []
final_BACC_collection_test = []
final_Sn_collection_test = []
final_Sp_collection_test = []
final_MCC_collection_test = []
final_AUC_collection_test = []
for i in range(len(CNN_channel)):
  # split dataset ten times and each times as 8:2 ratio for data division
  ACC_collection_test = []
  BACC_collection_test = []
  Sn_collection_test = []
  Sp_collection_test = []
  MCC_collection_test = []
  AUC_collection_test = []
  for a in range(10):
      random_num = a
      X_train_whole, X_test, y_train_whole, y_test = train_test_split( X, y, test_size=0.2, random_state=random_num,shuffle=True, stratify = y)

      # train the model with the optimal parameters in the cross validation
      saved_model = train_model(X_train_whole, y_train_whole,CNN_channel[i],kernel_size[i],stride_size[i],dense_node[i]) # optimal paraemters
      # performance evaluation at the independent test dataset
      ACC, Sn, Sp, MCC, BACC, AUC = check_performance(saved_model, X_test, y_test)
      print(ACC, Sn, Sp, MCC, BACC, AUC)
      ACC_collection_test.append(ACC)
      BACC_collection_test.append(BACC)
      Sn_collection_test.append(Sn)
      Sp_collection_test.append(Sp)
      MCC_collection_test.append(MCC)
      AUC_collection_test.append(AUC)

  import math
  final_ACC_collection_test.append(str(round(statistics.mean(ACC_collection_test),3))+'±'+ str(round(statistics.stdev(ACC_collection_test),3)))
  if any(math.isnan(i) for i in MCC_collection_test):
      final_BACC_collection_test.append('none')
      final_Sn_collection_test.append('none')
      final_Sp_collection_test.append('none')
      final_MCC_collection_test.append('none')
      final_AUC_collection_test.append('none')
  else:
      final_BACC_collection_test.append(str(round(statistics.mean(BACC_collection_test),3))+'±'+str(round(statistics.stdev(BACC_collection_test),3)))
      final_Sn_collection_test.append(str(round(statistics.mean(Sn_collection_test),3))+'±'+str(round(statistics.stdev(Sn_collection_test),3)))
      final_Sp_collection_test.append(str(round(statistics.mean(Sp_collection_test),3))+'±'+str(round(statistics.stdev(Sp_collection_test),3)))
      final_MCC_collection_test.append(str(round(statistics.mean(MCC_collection_test),3))+'±'+str(round(statistics.stdev(MCC_collection_test),3)))
      final_AUC_collection_test.append(str(round(statistics.mean(AUC_collection_test),3))+'±'+str(round(statistics.stdev(AUC_collection_test),3)))

# performance in test
print(final_ACC_collection_test, '\n',BACC_collection_test, '\n',Sn_collection_test, '\n',Sp_collection_test, '\n',MCC_collection_test,'\n', AUC_collection_test)


### 640 feature dimension embedding test

#### defined function for model development and evaluation

In [ ]:
def train_model(X_train, X_valid, y_train, y_valid, filter_size,kernel_size,stride_size,node_units):
  inputShape=(640,1) # input feature size
  input = Input(inputShape)
  x = Conv1D(filter_size,(kernel_size),strides = (stride_size),name='layer_conv2',padding='same')(input)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = MaxPooling1D((2), name='MaxPool2',padding="same")(x)
  x = Dropout(0.15)(x)
  x = Flatten()(x)
  x = Dense(node_units,activation = 'relu',name='fc1')(x)
  x = Dropout(0.15)(x)
  x = Dense(2,activation = 'softmax',name='fc2')(x)
  model = Model(inputs = input,outputs = x,name='Predict')

  # define SGD optimizer
  momentum = 0.5
  sgd = SGD(learning_rate=0.01, momentum=momentum, nesterov=False)
  # compile the model
  model.compile(loss='sparse_categorical_crossentropy',optimizer=sgd, metrics=['accuracy'])
  # learning deccay setting
  import math
  def step_decay(epoch): # gradually decrease the learning rate
      initial_lrate=0.1
      drop=0.6
      epochs_drop = 3.0
      lrate= initial_lrate * math.pow(drop,    # math.pow base raised to a power
            math.floor((1+epoch)/epochs_drop)) # math.floor Round numbers down to the nearest integer
      return lrate
  lr = LearningRateScheduler(step_decay)
  # early stop setting
  early_stop = EarlyStopping(monitor='val_accuracy', patience = 20,verbose=0,restore_best_weights = True)
  # set checkpoint and save the best model
  mc = ModelCheckpoint('best_model_grid_640.h5',  monitor='val_accuracy', mode='max', verbose=0, save_best_only=True, save_weights_only=False)
  # summary the callbacks_list
  callbacks_list = [ lr , early_stop, mc]
  model_history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=200,callbacks=callbacks_list,batch_size = 32, verbose=1)
  # load the save best model
  saved_model = load_model('best_model_grid_640.h5')
  return saved_model

  # import gc
  # del model
  # del saved_model
  # import torch
  # import gc
  # torch.cuda.memory_reserved()
  # gc.collect()

def check_performance(saved_model, X_test, y_test):
  # result collection list
  # confusion matrix
  predicted_class= []
  predicted_protability = saved_model.predict(X_test,batch_size=1)
  for p in range(predicted_protability.shape[0]):
    index = np.where(predicted_protability[p] == np.amax(predicted_protability[p]))[0][0]
    predicted_class.append(index)
  predicted_class = np.array(predicted_class)
  y_true = y_test
  from sklearn.metrics import confusion_matrix
  import math
  # np.ravel() return a flatten 1D array
  TN, FP, FN, TP = confusion_matrix(y_true, predicted_class).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
  ACC = (TP+TN)/(TP+TN+FP+FN)

  Sn = (TP/(TP+FN))
  Sp = (TN/(TN+FP))
  MCC = (TP*TN-FP*FN)/math.pow(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)),0.5)
  BACC = (0.5*TP/(TP+FN)+0.5*TN/(TN+FP))
  from sklearn.metrics import roc_auc_score
  AUC = roc_auc_score(y_test, predicted_protability[:,1])
  return ACC, Sn, Sp, MCC, BACC, AUC

#### 5 Fold CV for optimal hyperparrameter

In [ ]:
import os
os.chdir('/content/drive/MyDrive/universal_allergenicity_new')

In [ ]:
import numpy as np
import pandas as pd
import esm
import statistics
# whole dataset loading and dataset splitting
dataset = pd.read_excel('allergens_dataset.xlsx',na_filter = False) # take care the NA sequence problem

# generate the peptide embeddings
sequence_list = dataset['sequence']
y = dataset['label']
y = np.array(y) # transformed as np.array for CNN model

# read the peptide embeddings
X_data_name = 'whole_sample_dataset_esm2_t30_150M_UR50D_unified_640_dimension.csv'
X_data = pd.read_csv(X_data_name,header=0, index_col = 0,delimiter=',')
X = np.array(X_data)
from sklearn.model_selection import train_test_split
random_num = 1
X_train_whole, X_test, y_train_whole, y_test = train_test_split( X, y, test_size=0.2, random_state=random_num,shuffle=True, stratify = y)


In [ ]:
# set the selected hyperparameters from above grid search
CNN_channel = [64, 64, 64, 32, 64, 16, 128, 64, 32, 32, 32, 64, 32, 16, 128, 16, 128, 64, 64, 64, 16, 64, 64, 16, 32, 128, 32, 16, 32, 64, 32, 64, 64, 64]
dense_node = [8192, 2048, 8192, 8192, 4096, 2048, 4096, 2048, 2048, 4096, 512, 1024, 4096, 2048, 4096, 1024, 512, 512, 1024, 512, 128, 1024, 512, 32, 32, 64, 64, 128, 128, 256, 256, 512, 512, 512]
kernel_size = [12, 12, 6, 9, 12, 12, 6, 9, 9, 9, 9, 9, 12, 9, 9, 12, 9, 9, 9, 12, 9, 6, 9, 12, 9, 12, 9, 9, 9, 9, 6, 9, 12, 9]
stride_size = [2, 4, 4, 2, 2, 2, 4, 2, 1, 2, 1, 2, 2, 2, 2, 2, 1, 1, 1, 2, 1, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2]
print(len(CNN_channel),len(dense_node), len(kernel_size),len(stride_size))

34 34 34 34


In [ ]:
import os
os.chdir('/content')

In [ ]:
# 5-Fold CV
final_ACC_collection_cv = []
final_BACC_collection_cv = []
final_Sn_collection_cv = []
final_Sp_collection_cv = []
final_MCC_collection_cv = []
final_AUC_collection_cv = []

final_ACC_collection_test = []
final_BACC_collection_test = []
final_Sn_collection_test = []
final_Sp_collection_test = []
final_MCC_collection_test = []
final_AUC_collection_test = []

# split dataset
for i in range(len(CNN_channel)):
  # collect the value in cross validation
  ACC_collection_cv = []
  BACC_collection_cv = []
  Sn_collection_cv = []
  Sp_collection_cv = []
  MCC_collection_cv = []
  AUC_collection_cv = []

  ACC_collection_test = []
  BACC_collection_test = []
  Sn_collection_test = []
  Sp_collection_test = []
  MCC_collection_test = []
  AUC_collection_test = []
  from sklearn.model_selection import StratifiedKFold
  kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

  for train_ix, test_ix in kfold.split(X_train_whole, y_train_whole):
      X_train, X_valid = X[train_ix], X[test_ix]
      y_train, y_valid = y[train_ix], y[test_ix]
      X_train_only, X_train_indicator, y_train_only, y_train_indicator = train_test_split( X_train, y_train, test_size=0.25, random_state=random_num,shuffle=True, stratify = y_train)

      # train the model
      saved_model = train_model(X_train_only, X_train_indicator, y_train_only, y_train_indicator,CNN_channel[i],kernel_size[i],stride_size[i],dense_node[i])

      ACC, Sn, Sp, MCC, BACC, AUC = check_performance(saved_model, X_valid, y_valid)
      print(ACC, Sn, Sp, MCC, BACC, AUC)
      ACC_collection_test.append(ACC)
      BACC_collection_test.append(BACC)
      Sn_collection_test.append(Sn)
      Sp_collection_test.append(Sp)
      MCC_collection_test.append(MCC)
      AUC_collection_test.append(AUC)
  import math
  final_ACC_collection_test.append(str(round(statistics.mean(ACC_collection_test),3))+'±'+ str(round(statistics.stdev(ACC_collection_test),3)))
  if any(math.isnan(i) for i in MCC_collection_test):
      final_BACC_collection_test.append('none')
      final_Sn_collection_test.append('none')
      final_Sp_collection_test.append('none')
      final_MCC_collection_test.append('none')
      final_AUC_collection_test.append('none')
  else:
      final_BACC_collection_test.append(str(round(statistics.mean(BACC_collection_test),3))+'±'+str(round(statistics.stdev(BACC_collection_test),3)))
      final_Sn_collection_test.append(str(round(statistics.mean(Sn_collection_test),3))+'±'+str(round(statistics.stdev(Sn_collection_test),3)))
      final_Sp_collection_test.append(str(round(statistics.mean(Sp_collection_test),3))+'±'+str(round(statistics.stdev(Sp_collection_test),3)))
      final_MCC_collection_test.append(str(round(statistics.mean(MCC_collection_test),3))+'±'+str(round(statistics.stdev(MCC_collection_test),3)))
      final_AUC_collection_test.append(str(round(statistics.mean(AUC_collection_test),3))+'±'+str(round(statistics.stdev(AUC_collection_test),3)))


Epoch 1/200
241/241 [==============================] - ETA: 0s - loss: 1.0890 - accuracy: 0.7200

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


241/241 [==============================] - 16s 14ms/step - loss: 1.0890 - accuracy: 0.7200 - val_loss: 0.3870 - val_accuracy: 0.8400 - lr: 0.1000
Epoch 2/200
241/241 [==============================] - 3s 13ms/step - loss: 0.3469 - accuracy: 0.8527 - val_loss: 0.2438 - val_accuracy: 0.8930 - lr: 0.1000
Epoch 3/200
241/241 [==============================] - 3s 14ms/step - loss: 0.2388 - accuracy: 0.8994 - val_loss: 0.2236 - val_accuracy: 0.8961 - lr: 0.0600
Epoch 4/200
241/241 [==============================] - 3s 13ms/step - loss: 0.2201 - accuracy: 0.9079 - val_loss: 0.2162 - val_accuracy: 0.9089 - lr: 0.0600
Epoch 5/200
241/241 [==============================] - 2s 7ms/step - loss: 0.2094 - accuracy: 0.9123 - val_loss: 0.2003 - val_accuracy: 0.9081 - lr: 0.0600
Epoch 6/200
241/241 [==============================] - 4s 16ms/step - loss: 0.1876 - accuracy: 0.9203 - val_loss: 0.2203 - val_accuracy: 0.9105 - lr: 0.0360
Epoch 7/200
241/241 [==============================] - 3s 12ms/step - 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


241/241 [==============================] - 4s 14ms/step - loss: 1.0837 - accuracy: 0.6953 - val_loss: 0.4660 - val_accuracy: 0.7510 - lr: 0.1000
Epoch 2/200
241/241 [==============================] - 4s 17ms/step - loss: 0.4717 - accuracy: 0.7844 - val_loss: 0.3706 - val_accuracy: 0.8732 - lr: 0.1000
Epoch 3/200
241/241 [==============================] - 2s 7ms/step - loss: 0.2957 - accuracy: 0.8722 - val_loss: 0.3132 - val_accuracy: 0.8681 - lr: 0.0600
Epoch 4/200
241/241 [==============================] - 4s 17ms/step - loss: 0.2597 - accuracy: 0.8896 - val_loss: 0.2425 - val_accuracy: 0.8961 - lr: 0.0600
Epoch 5/200
241/241 [==============================] - 2s 7ms/step - loss: 0.2374 - accuracy: 0.9027 - val_loss: 0.3194 - val_accuracy: 0.8626 - lr: 0.0600
Epoch 6/200
241/241 [==============================] - 4s 17ms/step - loss: 0.2100 - accuracy: 0.9131 - val_loss: 0.2288 - val_accuracy: 0.8981 - lr: 0.0360
Epoch 7/200
241/241 [==============================] - 3s 12ms/step - lo

...

241/241 [==============================] - 3s 9ms/step - loss: 1.0228 - accuracy: 0.6808 - val_loss: 0.5725 - val_accuracy: 0.7505 - lr: 0.1000
Epoch 2/200
241/241 [==============================] - 2s 8ms/step - loss: 0.4487 - accuracy: 0.7977 - val_loss: 0.3401 - val_accuracy: 0.8194 - lr: 0.1000
Epoch 3/200
241/241 [==============================] - 2s 8ms/step - loss: 0.2785 - accuracy: 0.8801 - val_loss: 0.2454 - val_accuracy: 0.8906 - lr: 0.0600
Epoch 4/200
241/241 [==============================] - 2s 10ms/step - loss: 0.2428 - accuracy: 0.8987 - val_loss: 0.2223 - val_accuracy: 0.9019 - lr: 0.0600
Epoch 5/200
241/241 [==============================] - 1s 5ms/step - loss: 0.2228 - accuracy: 0.9058 - val_loss: 0.2272 - val_accuracy: 0.8984 - lr: 0.0600
Epoch 6/200
241/241 [==============================] - 2s 8ms/step - loss: 0.1993 - accuracy: 0.9158 - val_loss: 0.2051 - val_accuracy: 0.9163 - lr: 0.0360
Epoch 7/200
241/241 [==============================] - 1s 5ms/step - loss: 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


241/241 [==============================] - 3s 9ms/step - loss: 2.9722 - accuracy: 0.6111 - val_loss: 0.6612 - val_accuracy: 0.6261 - lr: 0.1000
Epoch 2/200
241/241 [==============================] - 1s 5ms/step - loss: 0.6645 - accuracy: 0.6250 - val_loss: 0.6621 - val_accuracy: 0.6261 - lr: 0.1000
Epoch 3/200
241/241 [==============================] - 1s 5ms/step - loss: 0.6616 - accuracy: 0.6264 - val_loss: 0.6617 - val_accuracy: 0.6261 - lr: 0.0600
Epoch 4/200
241/241 [==============================] - 1s 5ms/step - loss: 0.6616 - accuracy: 0.6258 - val_loss: 0.6610 - val_accuracy: 0.6261 - lr: 0.0600
Epoch 5/200
241/241 [==============================] - 1s 5ms/step - loss: 0.6615 - accuracy: 0.6255 - val_loss: 0.6610 - val_accuracy: 0.6261 - lr: 0.0600
Epoch 6/200
241/241 [==============================] - 1s 5ms/step - loss: 0.6607 - accuracy: 0.6264 - val_loss: 0.6613 - val_accuracy: 0.6261 - lr: 0.0360
Epoch 7/200
241/241 [==============================] - 1s 5ms/step - loss: 0

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


241/241 [==============================] - 3s 9ms/step - loss: 2.0087 - accuracy: 0.6159 - val_loss: 0.6559 - val_accuracy: 0.6241 - lr: 0.1000
Epoch 2/200
241/241 [==============================] - 1s 5ms/step - loss: 0.6621 - accuracy: 0.6228 - val_loss: 0.6491 - val_accuracy: 0.6241 - lr: 0.1000
Epoch 3/200
241/241 [==============================] - 2s 8ms/step - loss: 0.6384 - accuracy: 0.6581 - val_loss: 0.6196 - val_accuracy: 0.6856 - lr: 0.0600
Epoch 4/200
241/241 [==============================] - 2s 10ms/step - loss: 0.6296 - accuracy: 0.6680 - val_loss: 0.6082 - val_accuracy: 0.6875 - lr: 0.0600
Epoch 5/200
241/241 [==============================] - 1s 5ms/step - loss: 0.6229 - accuracy: 0.6745 - val_loss: 0.6115 - val_accuracy: 0.6848 - lr: 0.0600
Epoch 6/200
241/241 [==============================] - 1s 5ms/step - loss: 0.6134 - accuracy: 0.6834 - val_loss: 0.6056 - val_accuracy: 0.6848 - lr: 0.0360
Epoch 7/200
241/241 [==============================] - 2s 8ms/step - loss: 

In [ ]:
# performance in test
print(final_ACC_collection_test, '\n',final_BACC_collection_test, '\n',final_Sn_collection_test, '\n',final_Sp_collection_test, '\n',final_MCC_collection_test,'\n', final_AUC_collection_test)

### final model

In [ ]:
# set the selected hyperparameters from above grid search
CNN_channel = [64,64,64,32,16,128,16,64,64]
dense_node = [8192,2048,8192,8192,2048,4096,2048,1024,1024]
kernel_size = [12,12,6,9,12,6,9,9,6]
stride_size = [2,4,4,2,2,4,2,1,4]
print(len(CNN_channel),len(dense_node), len(kernel_size),len(stride_size))

5 5 5 5


In [ ]:
def train_model(X_train, y_train, filter_size,kernel_size,stride_size,node_units):
  inputShape=(640,1) # input feature size
  input = Input(inputShape)
  x = Conv1D(filter_size,(kernel_size),strides = (stride_size),name='layer_conv2',padding='same')(input)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = MaxPooling1D((2), name='MaxPool2',padding="same")(x)
  x = Dropout(0.15)(x)
  x = Flatten()(x)
  x = Dense(node_units,activation = 'relu',name='fc1')(x)
  x = Dropout(0.15)(x)
  x = Dense(2,activation = 'softmax',name='fc2')(x)
  model = Model(inputs = input,outputs = x,name='Predict')

  # define SGD optimizer
  momentum = 0.5
  sgd = SGD(learning_rate=0.01, momentum=momentum, nesterov=False)
  # compile the model
  model.compile(loss='sparse_categorical_crossentropy',optimizer=sgd, metrics=['accuracy'])
  # learning deccay setting
  import math
  def step_decay(epoch): # gradually decrease the learning rate
      initial_lrate=0.1
      drop=0.6
      epochs_drop = 3.0
      lrate= initial_lrate * math.pow(drop,    # math.pow base raised to a power
            math.floor((1+epoch)/epochs_drop)) # math.floor Round numbers down to the nearest integer
      return lrate
  lr = LearningRateScheduler(step_decay)
   # summary the callbacks_list
  callbacks_list = [ lr ]
  model_history = model.fit(X_train, y_train,  epochs=45,callbacks=callbacks_list,batch_size = 32, verbose=1)
  # load the save best model
  return model


def check_performance(saved_model, X_test, y_test):
  # result collection list
  # confusion matrix
  predicted_class= []
  predicted_protability = saved_model.predict(X_test,batch_size=1)
  for p in range(predicted_protability.shape[0]):
    index = np.where(predicted_protability[p] == np.amax(predicted_protability[p]))[0][0]
    predicted_class.append(index)
  predicted_class = np.array(predicted_class)
  y_true = y_test
  from sklearn.metrics import confusion_matrix
  import math
  # np.ravel() return a flatten 1D array
  TN, FP, FN, TP = confusion_matrix(y_true, predicted_class).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
  ACC = (TP+TN)/(TP+TN+FP+FN)

  Sn = (TP/(TP+FN))
  Sp = (TN/(TN+FP))
  MCC = (TP*TN-FP*FN)/math.pow(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)),0.5)
  BACC = (0.5*TP/(TP+FN)+0.5*TN/(TN+FP))
  from sklearn.metrics import roc_auc_score
  AUC = roc_auc_score(y_test, predicted_protability[:,1])
  return ACC, Sn, Sp, MCC, BACC, AUC

In [ ]:
import os
os.chdir('/content/drive/MyDrive/universal_allergenicity_new')

In [ ]:
import numpy as np
import pandas as pd
import esm
import statistics
# whole dataset loading and dataset splitting
dataset = pd.read_excel('allergens_dataset.xlsx',na_filter = False) # take care the NA sequence problem

# generate the peptide embeddings
sequence_list = dataset['sequence']
y = dataset['label']
y = np.array(y) # transformed as np.array for CNN model

# read the peptide embeddings
X_data_name = 'whole_sample_dataset_esm2_t30_150M_UR50D_unified_640_dimension.csv'
X_data = pd.read_csv(X_data_name,header=0, index_col = 0,delimiter=',')
X = np.array(X_data)
from sklearn.model_selection import train_test_split



In [ ]:
import os
os.chdir('/content')

In [ ]:
# final model
final_ACC_collection_test = []
final_BACC_collection_test = []
final_Sn_collection_test = []
final_Sp_collection_test = []
final_MCC_collection_test = []
final_AUC_collection_test = []
for i in range(len(CNN_channel)):
  # split dataset ten times and each times as 8:2 ratio for data division
  ACC_collection_test = []
  BACC_collection_test = []
  Sn_collection_test = []
  Sp_collection_test = []
  MCC_collection_test = []
  AUC_collection_test = []
  for a in range(10):
      random_num = a
      X_train_whole, X_test, y_train_whole, y_test = train_test_split( X, y, test_size=0.2, random_state=random_num,shuffle=True, stratify = y)

      # train the model with the optimal parameters in the cross validation
      saved_model = train_model(X_train_whole, y_train_whole,CNN_channel[i],kernel_size[i],stride_size[i],dense_node[i]) # optimal paraemters
      # performance evaluation at the independent test dataset
      ACC, Sn, Sp, MCC, BACC, AUC = check_performance(saved_model, X_test, y_test)
      print(ACC, Sn, Sp, MCC, BACC, AUC)
      ACC_collection_test.append(ACC)
      BACC_collection_test.append(BACC)
      Sn_collection_test.append(Sn)
      Sp_collection_test.append(Sp)
      MCC_collection_test.append(MCC)
      AUC_collection_test.append(AUC)

  import math
  final_ACC_collection_test.append(str(round(statistics.mean(ACC_collection_test),3))+'±'+ str(round(statistics.stdev(ACC_collection_test),3)))
  if any(math.isnan(i) for i in MCC_collection_test):
      final_BACC_collection_test.append('none')
      final_Sn_collection_test.append('none')
      final_Sp_collection_test.append('none')
      final_MCC_collection_test.append('none')
      final_AUC_collection_test.append('none')
  else:
      final_BACC_collection_test.append(str(round(statistics.mean(BACC_collection_test),3))+'±'+str(round(statistics.stdev(BACC_collection_test),3)))
      final_Sn_collection_test.append(str(round(statistics.mean(Sn_collection_test),3))+'±'+str(round(statistics.stdev(Sn_collection_test),3)))
      final_Sp_collection_test.append(str(round(statistics.mean(Sp_collection_test),3))+'±'+str(round(statistics.stdev(Sp_collection_test),3)))
      final_MCC_collection_test.append(str(round(statistics.mean(MCC_collection_test),3))+'±'+str(round(statistics.stdev(MCC_collection_test),3)))
      final_AUC_collection_test.append(str(round(statistics.mean(AUC_collection_test),3))+'±'+str(round(statistics.stdev(AUC_collection_test),3)))

# performance in test
print(final_ACC_collection_test, '\n',BACC_collection_test, '\n',Sn_collection_test, '\n',Sp_collection_test, '\n',MCC_collection_test,'\n', AUC_collection_test)


### 1280 feature dimension embedding test

#### defined function for model development and evaluation

In [ ]:
def train_model(X_train, X_valid, y_train, y_valid, filter_size,kernel_size,stride_size,node_units):
  inputShape=(1280,1) # input feature size
  input = Input(inputShape)
  x = Conv1D(filter_size,(kernel_size),strides = (stride_size),name='layer_conv2',padding='same')(input)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = MaxPooling1D((2), name='MaxPool2',padding="same")(x)
  x = Dropout(0.15)(x)
  x = Flatten()(x)
  x = Dense(node_units,activation = 'relu',name='fc1')(x)
  x = Dropout(0.15)(x)
  x = Dense(2,activation = 'softmax',name='fc2')(x)
  model = Model(inputs = input,outputs = x,name='Predict')

  # define SGD optimizer
  momentum = 0.5
  sgd = SGD(learning_rate=0.01, momentum=momentum, nesterov=False)
  # compile the model
  model.compile(loss='sparse_categorical_crossentropy',optimizer=sgd, metrics=['accuracy'])
  # learning deccay setting
  import math
  def step_decay(epoch): # gradually decrease the learning rate
      initial_lrate=0.1
      drop=0.6
      epochs_drop = 3.0
      lrate= initial_lrate * math.pow(drop,    # math.pow base raised to a power
            math.floor((1+epoch)/epochs_drop)) # math.floor Round numbers down to the nearest integer
      return lrate
  lr = LearningRateScheduler(step_decay)
  # early stop setting
  early_stop = EarlyStopping(monitor='val_accuracy', patience = 20,verbose=0,restore_best_weights = True)
  # set checkpoint and save the best model
  mc = ModelCheckpoint('best_model_grid_1280.h5',  monitor='val_accuracy', mode='max', verbose=0, save_best_only=True, save_weights_only=False)
  # summary the callbacks_list
  callbacks_list = [ lr , early_stop, mc]
  model_history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=200,callbacks=callbacks_list,batch_size = 32, verbose=1)
  # load the save best model
  saved_model = load_model('best_model_grid_1280.h5')
  return saved_model

  # import gc
  # del model
  # del saved_model
  # import torch
  # import gc
  # torch.cuda.memory_reserved()
  # gc.collect()

def check_performance(saved_model, X_test, y_test):
  # result collection list
  # confusion matrix
  predicted_class= []
  predicted_protability = saved_model.predict(X_test,batch_size=1)
  for p in range(predicted_protability.shape[0]):
    index = np.where(predicted_protability[p] == np.amax(predicted_protability[p]))[0][0]
    predicted_class.append(index)
  predicted_class = np.array(predicted_class)
  y_true = y_test
  from sklearn.metrics import confusion_matrix
  import math
  # np.ravel() return a flatten 1D array
  TN, FP, FN, TP = confusion_matrix(y_true, predicted_class).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
  ACC = (TP+TN)/(TP+TN+FP+FN)

  Sn = (TP/(TP+FN))
  Sp = (TN/(TN+FP))
  MCC = (TP*TN-FP*FN)/math.pow(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)),0.5)
  BACC = (0.5*TP/(TP+FN)+0.5*TN/(TN+FP))
  from sklearn.metrics import roc_auc_score
  AUC = roc_auc_score(y_test, predicted_protability[:,1])
  return ACC, Sn, Sp, MCC, BACC, AUC

#### 5 Fold CV for optimal hyperparrameter

In [ ]:
import os
os.chdir('/content/drive/MyDrive/universal_allergenicity_new')

In [ ]:
import numpy as np
import pandas as pd
import esm
import statistics
# whole dataset loading and dataset splitting
dataset = pd.read_excel('allergens_dataset.xlsx',na_filter = False) # take care the NA sequence problem

# generate the peptide embeddings
sequence_list = dataset['sequence']
y = dataset['label']
y = np.array(y) # transformed as np.array for CNN model

# read the peptide embeddings
X_data_name = 'whole_sample_dataset_esm2_t33_650M_UR50D_unified_1280_dimension.csv'
X_data = pd.read_csv(X_data_name,header=0, index_col = 0,delimiter=',')
X = np.array(X_data)
from sklearn.model_selection import train_test_split
random_num = 1
X_train_whole, X_test, y_train_whole, y_test = train_test_split( X, y, test_size=0.2, random_state=random_num,shuffle=True, stratify = y)


In [ ]:
# set the selected hyperparameters from above grid search
CNN_channel = [32, 128, 32, 32, 32, 16, 64, 16, 16, 64, 64, 64, 43, 128, 64, 16, 128, 64, 128, 16, 128, 16, 64, 16, 16, 256, 32, 16, 32, 128, 16, 32, 32, 64, 256, 128, 128, 16, 64, 32, 128, 256]
dense_node = [8192, 4096, 1024, 8192, 2048, 1024, 4096, 512, 8192, 2048, 512, 256, 512, 1024, 1024, 2048, 4096, 1024, 512, 128, 1024, 1024, 512, 256, 32, 32, 32, 64, 64, 64, 128, 128, 128, 256, 256, 256, 256, 512, 512, 512, 512, 256]
kernel_size = [9, 12, 9, 9, 6, 12, 9, 12, 12, 9, 9, 12, 6, 12, 9, 12, 12, 12, 12, 12, 12, 6, 12, 3, 12, 6, 12, 6, 12, 12, 12, 12, 12, 12, 9, 9, 9, 12, 9, 6, 12, 9]
stride_size = [2, 8, 8, 8, 1, 2, 2, 1, 2, 2, 8, 2, 8, 1, 2, 2, 2, 4, 8, 1, 2, 4, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 1, 8, 2, 1, 8, 8, 8, 1]
print(len(CNN_channel),len(dense_node), len(kernel_size),len(stride_size))

42 42 42 42


In [ ]:
import os
os.chdir('/content')

In [ ]:
# 5-Fold CV
final_ACC_collection_cv = []
final_BACC_collection_cv = []
final_Sn_collection_cv = []
final_Sp_collection_cv = []
final_MCC_collection_cv = []
final_AUC_collection_cv = []

final_ACC_collection_test = []
final_BACC_collection_test = []
final_Sn_collection_test = []
final_Sp_collection_test = []
final_MCC_collection_test = []
final_AUC_collection_test = []

# split dataset
for a in range(len(CNN_channel)):
  i = a + 5
  # collect the value in cross validation
  ACC_collection_cv = []
  BACC_collection_cv = []
  Sn_collection_cv = []
  Sp_collection_cv = []
  MCC_collection_cv = []
  AUC_collection_cv = []

  ACC_collection_test = []
  BACC_collection_test = []
  Sn_collection_test = []
  Sp_collection_test = []
  MCC_collection_test = []
  AUC_collection_test = []
  from sklearn.model_selection import StratifiedKFold
  kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

  for train_ix, test_ix in kfold.split(X_train_whole, y_train_whole):
      X_train, X_valid = X[train_ix], X[test_ix]
      y_train, y_valid = y[train_ix], y[test_ix]
      X_train_only, X_train_indicator, y_train_only, y_train_indicator = train_test_split( X_train, y_train, test_size=0.25, random_state=random_num,shuffle=True, stratify = y_train)

      # train the model
      # saved_model = train_model(X_train_only, X_train_indicator, y_train_only, y_train_indicator,CNN_channel[i],kernel_size[i],stride_size[i],dense_node[i])
      saved_model = train_model(X_train_only, X_train_indicator, y_train_only, y_train_indicator,128,9,2,256)

      ACC, Sn, Sp, MCC, BACC, AUC = check_performance(saved_model, X_valid, y_valid)
      print(ACC, Sn, Sp, MCC, BACC, AUC)
      ACC_collection_test.append(ACC)
      BACC_collection_test.append(BACC)
      Sn_collection_test.append(Sn)
      Sp_collection_test.append(Sp)
      MCC_collection_test.append(MCC)
      AUC_collection_test.append(AUC)

  import math
  final_ACC_collection_test.append(str(round(statistics.mean(ACC_collection_test),3))+'±'+ str(round(statistics.stdev(ACC_collection_test),3)))
  if any(math.isnan(i) for i in MCC_collection_test):
      final_BACC_collection_test.append('none')
      final_Sn_collection_test.append('none')
      final_Sp_collection_test.append('none')
      final_MCC_collection_test.append('none')
      final_AUC_collection_test.append('none')
  else:
      final_BACC_collection_test.append(str(round(statistics.mean(BACC_collection_test),3))+'±'+str(round(statistics.stdev(BACC_collection_test),3)))
      final_Sn_collection_test.append(str(round(statistics.mean(Sn_collection_test),3))+'±'+str(round(statistics.stdev(Sn_collection_test),3)))
      final_Sp_collection_test.append(str(round(statistics.mean(Sp_collection_test),3))+'±'+str(round(statistics.stdev(Sp_collection_test),3)))
      final_MCC_collection_test.append(str(round(statistics.mean(MCC_collection_test),3))+'±'+str(round(statistics.stdev(MCC_collection_test),3)))
      final_AUC_collection_test.append(str(round(statistics.mean(AUC_collection_test),3))+'±'+str(round(statistics.stdev(AUC_collection_test),3)))


Epoch 1/200


KeyboardInterrupt: ignored

Epoch 1/200
241/241 [==============================] - 7s 11ms/step - loss: 0.9252 - accuracy: 0.7964 - val_loss: 0.4502 - val_accuracy: 0.8295 - lr: 0.1000
Epoch 2/200
  1/241 [..............................] - ETA: 2s - loss: 0.3719 - accuracy: 0.8750

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


241/241 [==============================] - 2s 10ms/step - loss: 0.2314 - accuracy: 0.8993 - val_loss: 0.2369 - val_accuracy: 0.9023 - lr: 0.1000
Epoch 3/200
241/241 [==============================] - 2s 10ms/step - loss: 0.1850 - accuracy: 0.9193 - val_loss: 0.1866 - val_accuracy: 0.9190 - lr: 0.0600
Epoch 4/200
241/241 [==============================] - 2s 10ms/step - loss: 0.1730 - accuracy: 0.9253 - val_loss: 0.1794 - val_accuracy: 0.9221 - lr: 0.0600
Epoch 5/200
241/241 [==============================] - 2s 10ms/step - loss: 0.1588 - accuracy: 0.9276 - val_loss: 0.1765 - val_accuracy: 0.9249 - lr: 0.0600
Epoch 6/200
241/241 [==============================] - 2s 10ms/step - loss: 0.1416 - accuracy: 0.9350 - val_loss: 0.1761 - val_accuracy: 0.9292 - lr: 0.0360
Epoch 7/200
241/241 [==============================] - 2s 8ms/step - loss: 0.1319 - accuracy: 0.9424 - val_loss: 0.1793 - val_accuracy: 0.9264 - lr: 0.0360
Epoch 8/200
241/241 [==============================] - 2s 9ms/step - lo

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


241/241 [==============================] - 3s 11ms/step - loss: nan - accuracy: 0.3841 - val_loss: nan - val_accuracy: 0.3739 - lr: 0.1000
Epoch 2/200
241/241 [==============================] - 2s 8ms/step - loss: nan - accuracy: 0.3737 - val_loss: nan - val_accuracy: 0.3739 - lr: 0.1000
Epoch 3/200
241/241 [==============================] - 2s 8ms/step - loss: nan - accuracy: 0.3737 - val_loss: nan - val_accuracy: 0.3739 - lr: 0.0600
Epoch 4/200
241/241 [==============================] - 2s 8ms/step - loss: nan - accuracy: 0.3737 - val_loss: nan - val_accuracy: 0.3739 - lr: 0.0600
Epoch 5/200
241/241 [==============================] - 2s 8ms/step - loss: nan - accuracy: 0.3737 - val_loss: nan - val_accuracy: 0.3739 - lr: 0.0600
Epoch 6/200
241/241 [==============================] - 2s 8ms/step - loss: nan - accuracy: 0.3737 - val_loss: nan - val_accuracy: 0.3739 - lr: 0.0360
Epoch 7/200
241/241 [==============================] - 2s 8ms/step - loss: nan - accuracy: 0.3737 - val_loss: n

IndexError: ignored

In [ ]:
# performance in test
print(final_ACC_collection_test, '\n',final_BACC_collection_test, '\n',final_Sn_collection_test, '\n',final_Sp_collection_test, '\n',final_MCC_collection_test,'\n', final_AUC_collection_test)

#### final model

In [ ]:
# set the selected hyperparameters from above grid search
CNN_channel = [128,32,32,16,64,64,128,16,32]
dense_node = [4096,1024,8192,8192,2048,1024,1024,1024,512]
kernel_size = [12,9,9,12,9,12,12,6,6]
stride_size = [8,8,8,2,2,4,2,4,8]
print(len(CNN_channel),len(dense_node), len(kernel_size),len(stride_size))

5 5 5 5


In [ ]:
def train_model(X_train, y_train, filter_size,kernel_size,stride_size,node_units):
  inputShape=(1280,1) # input feature size
  input = Input(inputShape)
  x = Conv1D(filter_size,(kernel_size),strides = (stride_size),name='layer_conv2',padding='same')(input)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = MaxPooling1D((2), name='MaxPool2',padding="same")(x)
  x = Dropout(0.15)(x)
  x = Flatten()(x)
  x = Dense(node_units,activation = 'relu',name='fc1')(x)
  x = Dropout(0.15)(x)
  x = Dense(2,activation = 'softmax',name='fc2')(x)
  model = Model(inputs = input,outputs = x,name='Predict')

  # define SGD optimizer
  momentum = 0.5
  sgd = SGD(learning_rate=0.01, momentum=momentum, nesterov=False)
  # compile the model
  model.compile(loss='sparse_categorical_crossentropy',optimizer=sgd, metrics=['accuracy'])
  # learning deccay setting
  import math
  def step_decay(epoch): # gradually decrease the learning rate
      initial_lrate=0.1
      drop=0.6
      epochs_drop = 3.0
      lrate= initial_lrate * math.pow(drop,    # math.pow base raised to a power
            math.floor((1+epoch)/epochs_drop)) # math.floor Round numbers down to the nearest integer
      return lrate
  lr = LearningRateScheduler(step_decay)
   # summary the callbacks_list
  callbacks_list = [ lr ]
  model_history = model.fit(X_train, y_train,  epochs=45,callbacks=callbacks_list,batch_size = 32, verbose=1)
  # load the save best model
  return model


def check_performance(saved_model, X_test, y_test):
  # result collection list
  # confusion matrix
  predicted_class= []
  predicted_protability = saved_model.predict(X_test,batch_size=1)
  for p in range(predicted_protability.shape[0]):
    index = np.where(predicted_protability[p] == np.amax(predicted_protability[p]))[0][0]
    predicted_class.append(index)
  predicted_class = np.array(predicted_class)
  y_true = y_test
  from sklearn.metrics import confusion_matrix
  import math
  # np.ravel() return a flatten 1D array
  TN, FP, FN, TP = confusion_matrix(y_true, predicted_class).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
  ACC = (TP+TN)/(TP+TN+FP+FN)

  Sn = (TP/(TP+FN))
  Sp = (TN/(TN+FP))
  MCC = (TP*TN-FP*FN)/math.pow(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)),0.5)
  BACC = (0.5*TP/(TP+FN)+0.5*TN/(TN+FP))
  from sklearn.metrics import roc_auc_score
  AUC = roc_auc_score(y_test, predicted_protability[:,1])
  return ACC, Sn, Sp, MCC, BACC, AUC

In [ ]:
import os
os.chdir('/content/drive/MyDrive/universal_allergenicity_new')

In [ ]:
import numpy as np
import pandas as pd
import esm
import statistics
# whole dataset loading and dataset splitting
dataset = pd.read_excel('allergens_dataset.xlsx',na_filter = False) # take care the NA sequence problem

# generate the peptide embeddings
sequence_list = dataset['sequence']
y = dataset['label']
y = np.array(y) # transformed as np.array for CNN model

# read the peptide embeddings
X_data_name = 'whole_sample_dataset_esm2_t33_650M_UR50D_unified_1280_dimension.csv'
X_data = pd.read_csv(X_data_name,header=0, index_col = 0,delimiter=',')
X = np.array(X_data)
from sklearn.model_selection import train_test_split



In [ ]:
import os
os.chdir('/content')

In [ ]:
# final model
final_ACC_collection_test = []
final_BACC_collection_test = []
final_Sn_collection_test = []
final_Sp_collection_test = []
final_MCC_collection_test = []
final_AUC_collection_test = []
for i in range(len(CNN_channel)):
  # split dataset ten times and each times as 8:2 ratio for data division
  ACC_collection_test = []
  BACC_collection_test = []
  Sn_collection_test = []
  Sp_collection_test = []
  MCC_collection_test = []
  AUC_collection_test = []
  for a in range(10):
      random_num = a
      X_train_whole, X_test, y_train_whole, y_test = train_test_split( X, y, test_size=0.2, random_state=random_num,shuffle=True, stratify = y)

      # train the model with the optimal parameters in the cross validation
      saved_model = train_model(X_train_whole, y_train_whole,CNN_channel[i],kernel_size[i],stride_size[i],dense_node[i]) # optimal paraemters
      # performance evaluation at the independent test dataset
      ACC, Sn, Sp, MCC, BACC, AUC = check_performance(saved_model, X_test, y_test)
      print(ACC, Sn, Sp, MCC, BACC, AUC)
      ACC_collection_test.append(ACC)
      BACC_collection_test.append(BACC)
      Sn_collection_test.append(Sn)
      Sp_collection_test.append(Sp)
      MCC_collection_test.append(MCC)
      AUC_collection_test.append(AUC)

  import math
  final_ACC_collection_test.append(str(round(statistics.mean(ACC_collection_test),3))+'±'+ str(round(statistics.stdev(ACC_collection_test),3)))
  if any(math.isnan(i) for i in MCC_collection_test):
      final_BACC_collection_test.append('none')
      final_Sn_collection_test.append('none')
      final_Sp_collection_test.append('none')
      final_MCC_collection_test.append('none')
      final_AUC_collection_test.append('none')
  else:
      final_BACC_collection_test.append(str(round(statistics.mean(BACC_collection_test),3))+'±'+str(round(statistics.stdev(BACC_collection_test),3)))
      final_Sn_collection_test.append(str(round(statistics.mean(Sn_collection_test),3))+'±'+str(round(statistics.stdev(Sn_collection_test),3)))
      final_Sp_collection_test.append(str(round(statistics.mean(Sp_collection_test),3))+'±'+str(round(statistics.stdev(Sp_collection_test),3)))
      final_MCC_collection_test.append(str(round(statistics.mean(MCC_collection_test),3))+'±'+str(round(statistics.stdev(MCC_collection_test),3)))
      final_AUC_collection_test.append(str(round(statistics.mean(AUC_collection_test),3))+'±'+str(round(statistics.stdev(AUC_collection_test),3)))

# performance in test
print(final_ACC_collection_test, '\n',BACC_collection_test, '\n',Sn_collection_test, '\n',Sp_collection_test, '\n',MCC_collection_test,'\n', AUC_collection_test)


Epoch 1/45
402/402 [==============================] - 6s 4ms/step - loss: 0.6932 - accuracy: 0.8106 - lr: 0.1000
Epoch 2/45
402/402 [==============================] - 2s 4ms/step - loss: 0.2737 - accuracy: 0.8842 - lr: 0.1000
Epoch 3/45
402/402 [==============================] - 2s 4ms/step - loss: 0.2359 - accuracy: 0.8999 - lr: 0.0600
Epoch 4/45
402/402 [==============================] - 2s 4ms/step - loss: 0.2206 - accuracy: 0.9081 - lr: 0.0600
Epoch 5/45
402/402 [==============================] - 2s 4ms/step - loss: 0.2108 - accuracy: 0.9126 - lr: 0.0600
Epoch 6/45
402/402 [==============================] - 1s 4ms/step - loss: 0.1892 - accuracy: 0.9222 - lr: 0.0360
Epoch 7/45
402/402 [==============================] - 1s 4ms/step - loss: 0.1859 - accuracy: 0.9232 - lr: 0.0360
Epoch 8/45
402/402 [==============================] - 1s 4ms/step - loss: 0.1786 - accuracy: 0.9269 - lr: 0.0360
Epoch 9/45
402/402 [==============================] - 1s 4ms/step - loss: 0.1667 - accuracy: 0.9

#### final model round 2 from 10 repeated times hyperparameters

In [ ]:
# set the selected hyperparameters from above grid search
CNN_channel = [128,32,32,64,128,16]
dense_node = [4096,1024,8192,1024,4096,512]
kernel_size = [12,9,9,9,12,12]
stride_size = [8,8,8,2,2,1]
print(len(CNN_channel),len(dense_node), len(kernel_size),len(stride_size))

6 6 6 6


In [ ]:
def train_model(X_train, y_train, filter_size,kernel_size,stride_size,node_units):
  inputShape=(1280,1) # input feature size
  input = Input(inputShape)
  x = Conv1D(filter_size,(kernel_size),strides = (stride_size),name='layer_conv2',padding='same')(input)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = MaxPooling1D((2), name='MaxPool2',padding="same")(x)
  x = Dropout(0.15)(x)
  x = Flatten()(x)
  x = Dense(node_units,activation = 'relu',name='fc1')(x)
  x = Dropout(0.15)(x)
  x = Dense(2,activation = 'softmax',name='fc2')(x)
  model = Model(inputs = input,outputs = x,name='Predict')

  # define SGD optimizer
  momentum = 0.5
  sgd = SGD(learning_rate=0.01, momentum=momentum, nesterov=False)
  # compile the model
  model.compile(loss='sparse_categorical_crossentropy',optimizer=sgd, metrics=['accuracy'])
  # learning deccay setting
  import math
  def step_decay(epoch): # gradually decrease the learning rate
      initial_lrate=0.1
      drop=0.6
      epochs_drop = 3.0
      lrate= initial_lrate * math.pow(drop,    # math.pow base raised to a power
            math.floor((1+epoch)/epochs_drop)) # math.floor Round numbers down to the nearest integer
      return lrate
  lr = LearningRateScheduler(step_decay)
   # summary the callbacks_list
  callbacks_list = [ lr ]
  model_history = model.fit(X_train, y_train,  epochs=45,callbacks=callbacks_list,batch_size = 32, verbose=1)
  # load the save best model
  return model


def check_performance(saved_model, X_test, y_test):
  # result collection list
  # confusion matrix
  predicted_class= []
  predicted_protability = saved_model.predict(X_test,batch_size=1)
  for p in range(predicted_protability.shape[0]):
    index = np.where(predicted_protability[p] == np.amax(predicted_protability[p]))[0][0]
    predicted_class.append(index)
  predicted_class = np.array(predicted_class)
  y_true = y_test
  from sklearn.metrics import confusion_matrix
  import math
  # np.ravel() return a flatten 1D array
  TN, FP, FN, TP = confusion_matrix(y_true, predicted_class).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
  ACC = (TP+TN)/(TP+TN+FP+FN)

  Sn = (TP/(TP+FN))
  Sp = (TN/(TN+FP))
  MCC = (TP*TN-FP*FN)/math.pow(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)),0.5)
  BACC = (0.5*TP/(TP+FN)+0.5*TN/(TN+FP))
  from sklearn.metrics import roc_auc_score
  AUC = roc_auc_score(y_test, predicted_protability[:,1])
  return ACC, Sn, Sp, MCC, BACC, AUC

In [ ]:
import os
os.chdir('/content/drive/MyDrive/universal_allergenicity_new')

In [ ]:
import numpy as np
import pandas as pd
import esm
import statistics
# whole dataset loading and dataset splitting
dataset = pd.read_excel('allergens_dataset.xlsx',na_filter = False) # take care the NA sequence problem

# generate the peptide embeddings
sequence_list = dataset['sequence']
y = dataset['label']
y = np.array(y) # transformed as np.array for CNN model

# read the peptide embeddings
X_data_name = 'whole_sample_dataset_esm2_t33_650M_UR50D_unified_1280_dimension.csv'
X_data = pd.read_csv(X_data_name,header=0, index_col = 0,delimiter=',')
X = np.array(X_data)
from sklearn.model_selection import train_test_split



In [ ]:
import os
os.chdir('/content')

In [ ]:
# final model
final_ACC_collection_test = []
final_BACC_collection_test = []
final_Sn_collection_test = []
final_Sp_collection_test = []
final_MCC_collection_test = []
final_AUC_collection_test = []
for i in range(len(CNN_channel)):
  i = i+5
  # split dataset ten times and each times as 8:2 ratio for data division
  ACC_collection_test = []
  BACC_collection_test = []
  Sn_collection_test = []
  Sp_collection_test = []
  MCC_collection_test = []
  AUC_collection_test = []
  for a in range(10):
      random_num = a
      X_train_whole, X_test, y_train_whole, y_test = train_test_split( X, y, test_size=0.2, random_state=random_num,shuffle=True, stratify = y)

      # train the model with the optimal parameters in the cross validation
      saved_model = train_model(X_train_whole, y_train_whole,CNN_channel[i],kernel_size[i],stride_size[i],dense_node[i]) # optimal paraemters
      # performance evaluation at the independent test dataset
      ACC, Sn, Sp, MCC, BACC, AUC = check_performance(saved_model, X_test, y_test)
      print(ACC, Sn, Sp, MCC, BACC, AUC)
      ACC_collection_test.append(ACC)
      BACC_collection_test.append(BACC)
      Sn_collection_test.append(Sn)
      Sp_collection_test.append(Sp)
      MCC_collection_test.append(MCC)
      AUC_collection_test.append(AUC)

  import math
  final_ACC_collection_test.append(str(round(statistics.mean(ACC_collection_test),3))+'±'+ str(round(statistics.stdev(ACC_collection_test),3)))
  if any(math.isnan(i) for i in MCC_collection_test):
      final_BACC_collection_test.append('none')
      final_Sn_collection_test.append('none')
      final_Sp_collection_test.append('none')
      final_MCC_collection_test.append('none')
      final_AUC_collection_test.append('none')
  else:
      final_BACC_collection_test.append(str(round(statistics.mean(BACC_collection_test),3))+'±'+str(round(statistics.stdev(BACC_collection_test),3)))
      final_Sn_collection_test.append(str(round(statistics.mean(Sn_collection_test),3))+'±'+str(round(statistics.stdev(Sn_collection_test),3)))
      final_Sp_collection_test.append(str(round(statistics.mean(Sp_collection_test),3))+'±'+str(round(statistics.stdev(Sp_collection_test),3)))
      final_MCC_collection_test.append(str(round(statistics.mean(MCC_collection_test),3))+'±'+str(round(statistics.stdev(MCC_collection_test),3)))
      final_AUC_collection_test.append(str(round(statistics.mean(AUC_collection_test),3))+'±'+str(round(statistics.stdev(AUC_collection_test),3)))

# performance in test
print(final_ACC_collection_test, '\n',final_BACC_collection_test, '\n',final_Sn_collection_test, '\n',final_Sp_collection_test, '\n',final_MCC_collection_test,'\n', final_AUC_collection_test)


Epoch 1/45
402/402 [==============================] - 3s 4ms/step - loss: 1.0568 - accuracy: 0.5045 - lr: 0.1000
Epoch 2/45
402/402 [==============================] - 2s 4ms/step - loss: 0.6944 - accuracy: 0.5061 - lr: 0.1000
Epoch 3/45
402/402 [==============================] - 2s 4ms/step - loss: 0.6909 - accuracy: 0.5193 - lr: 0.0600
Epoch 4/45
402/402 [==============================] - 2s 4ms/step - loss: 0.6851 - accuracy: 0.5483 - lr: 0.0600
Epoch 5/45
402/402 [==============================] - 2s 4ms/step - loss: 0.6774 - accuracy: 0.5488 - lr: 0.0600
Epoch 6/45
402/402 [==============================] - 2s 4ms/step - loss: 0.5393 - accuracy: 0.7087 - lr: 0.0360
Epoch 7/45
402/402 [==============================] - 2s 4ms/step - loss: 0.4438 - accuracy: 0.7759 - lr: 0.0360
Epoch 8/45
402/402 [==============================] - 1s 4ms/step - loss: 0.3378 - accuracy: 0.8451 - lr: 0.0360
Epoch 9/45
402/402 [==============================] - 2s 4ms/step - loss: 0.2948 - accuracy: 0.8

IndexError: ignored

In [ ]:
# performance in test
print(final_ACC_collection_test, '\n',final_BACC_collection_test, '\n',final_Sn_collection_test, '\n',final_Sp_collection_test, '\n',final_MCC_collection_test,'\n', final_AUC_collection_test)


['0.889±0.138'] 
 ['0.889±0.138'] 
 ['0.825±0.289'] 
 ['0.953±0.029'] 
 ['0.783±0.264'] 
 ['0.932±0.152']


In [ ]:
# performance in test
print(final_ACC_collection_test, '\n',final_BACC_collection_test, '\n',final_Sn_collection_test, '\n',final_Sp_collection_test, '\n',final_MCC_collection_test,'\n', final_AUC_collection_test)


['0.947±0.004', '0.946±0.005', '0.946±0.004', '0.904±0.141'] 
 ['0.947±0.004', '0.946±0.005', '0.946±0.004', '0.904±0.141'] 
 ['0.935±0.006', '0.931±0.009', '0.932±0.006', '0.845±0.295'] 
 ['0.959±0.004', '0.961±0.003', '0.961±0.008', '0.963±0.013'] 
 ['0.894±0.009', '0.892±0.01', '0.893±0.008', '0.81±0.275'] 
 ['0.988±0.001', '0.988±0.001', '0.988±0.001', '0.94±0.154']


### 2560 feature dimension embedding test

#### defined function for model development and evaluation

In [ ]:
def train_model(X_train, X_valid, y_train, y_valid, filter_size,kernel_size,stride_size,node_units):
  inputShape=(1280,1) # input feature size
  input = Input(inputShape)
  x = Conv1D(filter_size,(kernel_size),strides = (stride_size),name='layer_conv2',padding='same')(input)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = MaxPooling1D((2), name='MaxPool2',padding="same")(x)
  x = Dropout(0.15)(x)
  x = Flatten()(x)
  x = Dense(node_units,activation = 'relu',name='fc1')(x)
  x = Dropout(0.15)(x)
  x = Dense(2,activation = 'softmax',name='fc2')(x)
  model = Model(inputs = input,outputs = x,name='Predict')

  # define SGD optimizer
  momentum = 0.5
  sgd = SGD(learning_rate=0.01, momentum=momentum, nesterov=False)
  # compile the model
  model.compile(loss='sparse_categorical_crossentropy',optimizer=sgd, metrics=['accuracy'])
  # learning deccay setting
  import math
  def step_decay(epoch): # gradually decrease the learning rate
      initial_lrate=0.1
      drop=0.6
      epochs_drop = 3.0
      lrate= initial_lrate * math.pow(drop,    # math.pow base raised to a power
            math.floor((1+epoch)/epochs_drop)) # math.floor Round numbers down to the nearest integer
      return lrate
  lr = LearningRateScheduler(step_decay)
  # early stop setting
  early_stop = EarlyStopping(monitor='val_accuracy', patience = 20,verbose=0,restore_best_weights = True)
  # set checkpoint and save the best model
  mc = ModelCheckpoint('best_model_grid_2560.h5',  monitor='val_accuracy', mode='max', verbose=0, save_best_only=True, save_weights_only=False)
  # summary the callbacks_list
  callbacks_list = [ lr , early_stop, mc]
  model_history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=200,callbacks=callbacks_list,batch_size = 32, verbose=1)
  # load the save best model
  saved_model = load_model('best_model_grid_2560.h5')
  return saved_model

  # import gc
  # del model
  # del saved_model
  # import torch
  # import gc
  # torch.cuda.memory_reserved()
  # gc.collect()

def check_performance(saved_model, X_test, y_test):
  # result collection list
  # confusion matrix
  predicted_class= []
  predicted_protability = saved_model.predict(X_test,batch_size=1)
  for p in range(predicted_protability.shape[0]):
    index = np.where(predicted_protability[p] == np.amax(predicted_protability[p]))[0][0]
    predicted_class.append(index)
  predicted_class = np.array(predicted_class)
  y_true = y_test
  from sklearn.metrics import confusion_matrix
  import math
  # np.ravel() return a flatten 1D array
  TN, FP, FN, TP = confusion_matrix(y_true, predicted_class).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
  ACC = (TP+TN)/(TP+TN+FP+FN)

  Sn = (TP/(TP+FN))
  Sp = (TN/(TN+FP))
  MCC = (TP*TN-FP*FN)/math.pow(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)),0.5)
  BACC = (0.5*TP/(TP+FN)+0.5*TN/(TN+FP))
  from sklearn.metrics import roc_auc_score
  AUC = roc_auc_score(y_test, predicted_protability[:,1])
  return ACC, Sn, Sp, MCC, BACC, AUC

#### 5 Fold CV for optimal hyperparrameter

In [ ]:
import os
os.chdir('/content/drive/MyDrive/universal_allergenicity_new')

In [ ]:
import numpy as np
import pandas as pd
import esm
import statistics
# whole dataset loading and dataset splitting
dataset = pd.read_excel('allergens_dataset.xlsx',na_filter = False) # take care the NA sequence problem

# generate the peptide embeddings
sequence_list = dataset['sequence']
y = dataset['label']
y = np.array(y) # transformed as np.array for CNN model

# read the peptide embeddings
X_data_name = 'whole_sample_dataset_esm2_t36_3B_UR50D_unified_2560_dimension.csv'
X_data = pd.read_csv(X_data_name,header=0, index_col = 0,delimiter=',')
X = np.array(X_data)
from sklearn.model_selection import train_test_split
random_num = 1
X_train_whole, X_test, y_train_whole, y_test = train_test_split( X, y, test_size=0.2, random_state=random_num,shuffle=True, stratify = y)


In [ ]:
# set the selected hyperparameters from above grid search
CNN_channel = [128, 64, 64, 16, 64, 128, 32, 128, 64, 32, 32, 128, 64, 16, 32, 16, 32, 64, 16, 64, 32, 32, 16, 64, 32, 32, 64, 128, 256, 32, 32, 16, 16, 16, 32, 16, 16]
dense_node = [2048, 2048, 4096, 512, 4096, 1024, 2048, 4096, 1024, 256, 2048, 1024, 128, 4096, 4096, 32, 32, 32, 32, 64, 64, 64, 64, 128, 128, 128, 128, 256, 256, 256, 256, 256, 512, 512, 512, 512, 512]
kernel_size = [6, 9, 12, 6, 6, 9, 6, 12, 9, 12, 12, 6, 9, 6, 3, 6, 12, 9, 3, 3, 9, 9, 3, 6, 6, 6, 9, 6, 9, 9, 12, 3, 6, 6, 6, 6, 3]
stride_size = [4, 8, 8, 4, 4, 8, 8, 8, 1, 8, 8, 8, 1, 8, 4, 4, 2, 2, 4, 1, 8, 4, 2, 1, 4, 8, 1, 4, 1, 2, 8, 2, 4, 2, 2, 8, 2]
print(len(CNN_channel),len(dense_node), len(kernel_size),len(stride_size))

34 34 34 34


In [ ]:
import os
os.chdir('/content')

In [ ]:
# 5-Fold CV
final_ACC_collection_cv = []
final_BACC_collection_cv = []
final_Sn_collection_cv = []
final_Sp_collection_cv = []
final_MCC_collection_cv = []
final_AUC_collection_cv = []

final_ACC_collection_test = []
final_BACC_collection_test = []
final_Sn_collection_test = []
final_Sp_collection_test = []
final_MCC_collection_test = []
final_AUC_collection_test = []

# split dataset
for i in range(len(CNN_channel)):
  # collect the value in cross validation
  ACC_collection_cv = []
  BACC_collection_cv = []
  Sn_collection_cv = []
  Sp_collection_cv = []
  MCC_collection_cv = []
  AUC_collection_cv = []

  ACC_collection_test = []
  BACC_collection_test = []
  Sn_collection_test = []
  Sp_collection_test = []
  MCC_collection_test = []
  AUC_collection_test = []
  from sklearn.model_selection import StratifiedKFold
  kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

  for train_ix, test_ix in kfold.split(X_train_whole, y_train_whole):
      X_train, X_valid = X[train_ix], X[test_ix]
      y_train, y_valid = y[train_ix], y[test_ix]
      X_train_only, X_train_indicator, y_train_only, y_train_indicator = train_test_split( X_train, y_train, test_size=0.25, random_state=random_num,shuffle=True, stratify = y_train)

      # train the model
      saved_model = train_model(X_train_only, X_train_indicator, y_train_only, y_train_indicator,CNN_channel[i],kernel_size[i],stride_size[i],dense_node[i])

      ACC, Sn, Sp, MCC, BACC, AUC = check_performance(saved_model, X_valid, y_valid)
      print(ACC, Sn, Sp, MCC, BACC, AUC)
      ACC_collection_test.append(ACC)
      BACC_collection_test.append(BACC)
      Sn_collection_test.append(Sn)
      Sp_collection_test.append(Sp)
      MCC_collection_test.append(MCC)
      AUC_collection_test.append(AUC)

  final_ACC_collection_test.append(str(round(statistics.mean(ACC_collection_test),3))+'±'+ str(round(statistics.stdev(ACC_collection_test),3)))
  final_BACC_collection_test.append(str(round(statistics.mean(BACC_collection_test),3))+'±'+str(round(statistics.stdev(BACC_collection_test),3)))
  final_Sn_collection_test.append(str(round(statistics.mean(Sn_collection_test),3))+'±'+str(round(statistics.stdev(Sn_collection_test),3)))
  final_Sp_collection_test.append(str(round(statistics.mean(Sp_collection_test),3))+'±'+str(round(statistics.stdev(Sp_collection_test),3)))
  final_MCC_collection_test.append(str(round(statistics.mean(MCC_collection_test),3))+'±'+str(round(statistics.stdev(MCC_collection_test),3)))
  final_AUC_collection_test.append(str(round(statistics.mean(AUC_collection_test),3))+'±'+str(round(statistics.stdev(AUC_collection_test),3)))


Epoch 1/200


In [ ]:
# performance in test
print(final_ACC_collection_test, '\n',final_BACC_collection_test, '\n',final_Sn_collection_test, '\n',final_Sp_collection_test, '\n',final_MCC_collection_test,'\n', final_AUC_collection_test)

### final model

In [ ]:
# set the selected hyperparameters from above grid search
CNN_channel = [32,16,16]
dense_node = [2048,32,512]
kernel_size = [6,6,6]
stride_size = [8,4,8]
print(len(CNN_channel),len(dense_node), len(kernel_size),len(stride_size))

3 3 3 3


In [ ]:
def train_model(X_train, y_train, filter_size,kernel_size,stride_size,node_units):
  inputShape=(2560,1) # input feature size
  input = Input(inputShape)
  x = Conv1D(filter_size,(kernel_size),strides = (stride_size),name='layer_conv2',padding='same')(input)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = MaxPooling1D((2), name='MaxPool2',padding="same")(x)
  x = Dropout(0.15)(x)
  x = Flatten()(x)
  x = Dense(node_units,activation = 'relu',name='fc1')(x)
  x = Dropout(0.15)(x)
  x = Dense(2,activation = 'softmax',name='fc2')(x)
  model = Model(inputs = input,outputs = x,name='Predict')

  # define SGD optimizer
  momentum = 0.5
  sgd = SGD(learning_rate=0.01, momentum=momentum, nesterov=False)
  # compile the model
  model.compile(loss='sparse_categorical_crossentropy',optimizer=sgd, metrics=['accuracy'])
  # learning deccay setting
  import math
  def step_decay(epoch): # gradually decrease the learning rate
      initial_lrate=0.1
      drop=0.6
      epochs_drop = 3.0
      lrate= initial_lrate * math.pow(drop,    # math.pow base raised to a power
            math.floor((1+epoch)/epochs_drop)) # math.floor Round numbers down to the nearest integer
      return lrate
  lr = LearningRateScheduler(step_decay)
   # summary the callbacks_list
  callbacks_list = [ lr ]
  model_history = model.fit(X_train, y_train,  epochs=45,callbacks=callbacks_list,batch_size = 32, verbose=1)
  # load the save best model
  return model


def check_performance(saved_model, X_test, y_test):
  # result collection list
  # confusion matrix
  predicted_class= []
  predicted_protability = saved_model.predict(X_test,batch_size=1)
  for p in range(predicted_protability.shape[0]):
    index = np.where(predicted_protability[p] == np.amax(predicted_protability[p]))[0][0]
    predicted_class.append(index)
  predicted_class = np.array(predicted_class)
  y_true = y_test
  from sklearn.metrics import confusion_matrix
  import math
  # np.ravel() return a flatten 1D array
  TN, FP, FN, TP = confusion_matrix(y_true, predicted_class).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
  ACC = (TP+TN)/(TP+TN+FP+FN)

  Sn = (TP/(TP+FN))
  Sp = (TN/(TN+FP))
  MCC = (TP*TN-FP*FN)/math.pow(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)),0.5)
  BACC = (0.5*TP/(TP+FN)+0.5*TN/(TN+FP))
  from sklearn.metrics import roc_auc_score
  AUC = roc_auc_score(y_test, predicted_protability[:,1])
  return ACC, Sn, Sp, MCC, BACC, AUC

In [ ]:
import os
os.chdir('/content/drive/MyDrive/universal_allergenicity_new')

In [ ]:
import numpy as np
import pandas as pd
import esm
import statistics
# whole dataset loading and dataset splitting
dataset = pd.read_excel('allergens_dataset.xlsx',na_filter = False) # take care the NA sequence problem

# generate the peptide embeddings
sequence_list = dataset['sequence']
y = dataset['label']
y = np.array(y) # transformed as np.array for CNN model

# read the peptide embeddings
X_data_name = 'whole_sample_dataset_esm2_t36_3B_UR50D_unified_2560_dimension.csv'
X_data = pd.read_csv(X_data_name,header=0, index_col = 0,delimiter=',')
X = np.array(X_data)
from sklearn.model_selection import train_test_split



In [ ]:
import os
os.chdir('/content')

In [ ]:
# final model
final_ACC_collection_test = []
final_BACC_collection_test = []
final_Sn_collection_test = []
final_Sp_collection_test = []
final_MCC_collection_test = []
final_AUC_collection_test = []
for i in range(len(CNN_channel)):
  # split dataset ten times and each times as 8:2 ratio for data division
  ACC_collection_test = []
  BACC_collection_test = []
  Sn_collection_test = []
  Sp_collection_test = []
  MCC_collection_test = []
  AUC_collection_test = []
  for a in range(10):
      random_num = a
      X_train_whole, X_test, y_train_whole, y_test = train_test_split( X, y, test_size=0.2, random_state=random_num,shuffle=True, stratify = y)

      # train the model with the optimal parameters in the cross validation
      saved_model = train_model(X_train_whole, y_train_whole,CNN_channel[i],kernel_size[i],stride_size[i],dense_node[i]) # optimal paraemters
      # performance evaluation at the independent test dataset
      ACC, Sn, Sp, MCC, BACC, AUC = check_performance(saved_model, X_test, y_test)
      print(ACC, Sn, Sp, MCC, BACC, AUC)
      ACC_collection_test.append(ACC)
      BACC_collection_test.append(BACC)
      Sn_collection_test.append(Sn)
      Sp_collection_test.append(Sp)
      MCC_collection_test.append(MCC)
      AUC_collection_test.append(AUC)

  import math
  final_ACC_collection_test.append(str(round(statistics.mean(ACC_collection_test),3))+'±'+ str(round(statistics.stdev(ACC_collection_test),3)))
  if any(math.isnan(i) for i in MCC_collection_test):
      final_BACC_collection_test.append('none')
      final_Sn_collection_test.append('none')
      final_Sp_collection_test.append('none')
      final_MCC_collection_test.append('none')
      final_AUC_collection_test.append('none')
  else:
      final_BACC_collection_test.append(str(round(statistics.mean(BACC_collection_test),3))+'±'+str(round(statistics.stdev(BACC_collection_test),3)))
      final_Sn_collection_test.append(str(round(statistics.mean(Sn_collection_test),3))+'±'+str(round(statistics.stdev(Sn_collection_test),3)))
      final_Sp_collection_test.append(str(round(statistics.mean(Sp_collection_test),3))+'±'+str(round(statistics.stdev(Sp_collection_test),3)))
      final_MCC_collection_test.append(str(round(statistics.mean(MCC_collection_test),3))+'±'+str(round(statistics.stdev(MCC_collection_test),3)))
      final_AUC_collection_test.append(str(round(statistics.mean(AUC_collection_test),3))+'±'+str(round(statistics.stdev(AUC_collection_test),3)))

# performance in test
print(final_ACC_collection_test, '\n',final_BACC_collection_test, '\n',final_Sn_collection_test, '\n',final_Sp_collection_test, '\n',final_MCC_collection_test,'\n', final_AUC_collection_test)


Epoch 1/45
402/402 [==============================] - 14s 5ms/step - loss: 0.2896 - accuracy: 0.8896 - lr: 0.1000
Epoch 2/45
402/402 [==============================] - 2s 5ms/step - loss: 0.1800 - accuracy: 0.9264 - lr: 0.1000
Epoch 3/45
402/402 [==============================] - 2s 5ms/step - loss: 0.1394 - accuracy: 0.9433 - lr: 0.0600
Epoch 4/45
402/402 [==============================] - 2s 5ms/step - loss: 0.1221 - accuracy: 0.9515 - lr: 0.0600
Epoch 5/45
402/402 [==============================] - 2s 6ms/step - loss: 0.1051 - accuracy: 0.9597 - lr: 0.0600
Epoch 6/45
402/402 [==============================] - 2s 5ms/step - loss: 0.0873 - accuracy: 0.9671 - lr: 0.0360
Epoch 7/45
402/402 [==============================] - 2s 5ms/step - loss: 0.0797 - accuracy: 0.9689 - lr: 0.0360
Epoch 8/45
402/402 [==============================] - 2s 5ms/step - loss: 0.0732 - accuracy: 0.9722 - lr: 0.0360
Epoch 9/45
402/402 [==============================] - 2s 5ms/step - loss: 0.0592 - accuracy: 0.

In [ ]:
# performance in test
print(final_ACC_collection_test, '\n',final_BACC_collection_test, '\n',final_Sn_collection_test, '\n',final_Sp_collection_test, '\n',final_MCC_collection_test,'\n', final_AUC_collection_test)


[] 
 [] 
 [] 
 [] 
 [] 
 []


In [ ]:
上面这个漏了第一二个，但是似乎第一组参数本身会报错
# set the selected hyperparameters from above grid search
CNN_channel = [64,128,32,16,16]
dense_node = [4096,1024,2048,32,512]
kernel_size = [12,9,6,6,6]
stride_size = [8,8,8,4,8]
print(len(CNN_channel),len(dense_node), len(kernel_size),len(stride_size))

SyntaxError: ignored

### final model round 2 from 10 repeated times hyperparameters

In [ ]:
# set best parameters
CNN_channel = [128, 64, 64, 16, 64, 128, 32, 128, 64, 32, 32, 128, 64, 16, 32, 16, 32, 64, 16, 64, 32, 32, 16, 64, 32, 32, 64, 128, 256, 32, 32, 16, 16, 16, 32, 16, 16]
dense_node = [2048, 2048, 4096, 512, 4096, 1024, 2048, 4096, 1024, 256, 2048, 1024, 128, 4096, 4096, 32, 32, 32, 32, 64, 64, 64, 64, 128, 128, 128, 128, 256, 256, 256, 256, 256, 512, 512, 512, 512, 512]
kernel_size = [6, 9, 12, 6, 6, 9, 6, 12, 9, 12, 12, 6, 9, 6, 3, 6, 12, 9, 3, 3, 9, 9, 3, 6, 6, 6, 9, 6, 9, 9, 12, 3, 6, 6, 6, 6, 3]
stride_size = [4, 8, 8, 4, 4, 8, 8, 8, 1, 8, 8, 8, 1, 8, 4, 4, 2, 2, 4, 1, 8, 4, 2, 1, 4, 8, 1, 4, 1, 2, 8, 2, 4, 2, 2, 8, 2]
print(len(CNN_channel),len(dense_node), len(kernel_size),len(stride_size))

37 37 37 37


In [ ]:
def train_model(X_train, y_train, filter_size,kernel_size,stride_size,node_units):
  inputShape=(2560,1) # input feature size
  input = Input(inputShape)
  x = Conv1D(filter_size,(kernel_size),strides = (stride_size),name='layer_conv2',padding='same')(input)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = MaxPooling1D((2), name='MaxPool2',padding="same")(x)
  x = Dropout(0.15)(x)
  x = Flatten()(x)
  x = Dense(node_units,activation = 'relu',name='fc1')(x)
  x = Dropout(0.15)(x)
  x = Dense(2,activation = 'softmax',name='fc2')(x)
  model = Model(inputs = input,outputs = x,name='Predict')

  # define SGD optimizer
  momentum = 0.5
  sgd = SGD(learning_rate=0.01, momentum=momentum, nesterov=False)
  # compile the model
  model.compile(loss='sparse_categorical_crossentropy',optimizer=sgd, metrics=['accuracy'])
  # learning deccay setting
  import math
  def step_decay(epoch): # gradually decrease the learning rate
      initial_lrate=0.1
      drop=0.6
      epochs_drop = 3.0
      lrate= initial_lrate * math.pow(drop,    # math.pow base raised to a power
            math.floor((1+epoch)/epochs_drop)) # math.floor Round numbers down to the nearest integer
      return lrate
  lr = LearningRateScheduler(step_decay)
   # summary the callbacks_list
  callbacks_list = [ lr ]
  model_history = model.fit(X_train, y_train,  epochs=45,callbacks=callbacks_list,batch_size = 32, verbose=1)
  # load the save best model
  return model


def check_performance(saved_model, X_test, y_test):
  # result collection list
  # confusion matrix
  predicted_class= []
  predicted_protability = saved_model.predict(X_test,batch_size=1)
  for p in range(predicted_protability.shape[0]):
    index = np.where(predicted_protability[p] == np.amax(predicted_protability[p]))[0][0]
    predicted_class.append(index)
  predicted_class = np.array(predicted_class)
  y_true = y_test
  from sklearn.metrics import confusion_matrix
  import math
  # np.ravel() return a flatten 1D array
  TN, FP, FN, TP = confusion_matrix(y_true, predicted_class).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
  ACC = (TP+TN)/(TP+TN+FP+FN)

  Sn = (TP/(TP+FN))
  Sp = (TN/(TN+FP))
  MCC = (TP*TN-FP*FN)/math.pow(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)),0.5)
  BACC = (0.5*TP/(TP+FN)+0.5*TN/(TN+FP))
  from sklearn.metrics import roc_auc_score
  AUC = roc_auc_score(y_test, predicted_protability[:,1])
  return ACC, Sn, Sp, MCC, BACC, AUC

In [ ]:
import os
os.chdir('/content/drive/MyDrive/universal_allergenicity_new')

In [ ]:
import numpy as np
import pandas as pd
import esm
import statistics
# whole dataset loading and dataset splitting
dataset = pd.read_excel('allergens_dataset.xlsx',na_filter = False) # take care the NA sequence problem

# generate the peptide embeddings
sequence_list = dataset['sequence']
y = dataset['label']
y = np.array(y) # transformed as np.array for CNN model

# read the peptide embeddings
X_data_name = 'whole_sample_dataset_esm2_t36_3B_UR50D_unified_2560_dimension.csv'
X_data = pd.read_csv(X_data_name,header=0, index_col = 0,delimiter=',')
X = np.array(X_data)
from sklearn.model_selection import train_test_split



In [ ]:
import os
os.chdir('/content')

In [ ]:
# final model
final_ACC_collection_test = []
final_BACC_collection_test = []
final_Sn_collection_test = []
final_Sp_collection_test = []
final_MCC_collection_test = []
final_AUC_collection_test = []
for i in range(len(CNN_channel)):
  i= i + 2
  # split dataset ten times and each times as 8:2 ratio for data division
  ACC_collection_test = []
  BACC_collection_test = []
  Sn_collection_test = []
  Sp_collection_test = []
  MCC_collection_test = []
  AUC_collection_test = []
  for a in range(10):
      random_num = a
      X_train_whole, X_test, y_train_whole, y_test = train_test_split( X, y, test_size=0.2, random_state=random_num,shuffle=True)

      # train the model with the optimal parameters in the cross validation
      saved_model = train_model(X_train_whole, y_train_whole,CNN_channel[i],kernel_size[i],stride_size[i],dense_node[i]) # optimal paraemters
      # performance evaluation at the independent test dataset
      ACC, Sn, Sp, MCC, BACC, AUC = check_performance(saved_model, X_test, y_test)
      print(ACC, Sn, Sp, MCC, BACC, AUC)
      ACC_collection_test.append(ACC)
      BACC_collection_test.append(BACC)
      Sn_collection_test.append(Sn)
      Sp_collection_test.append(Sp)
      MCC_collection_test.append(MCC)
      AUC_collection_test.append(AUC)

  import math
  final_ACC_collection_test.append(str(round(statistics.mean(ACC_collection_test),3))+'±'+ str(round(statistics.stdev(ACC_collection_test),3)))
  if any(math.isnan(i) for i in MCC_collection_test):
      final_BACC_collection_test.append('none')
      final_Sn_collection_test.append('none')
      final_Sp_collection_test.append('none')
      final_MCC_collection_test.append('none')
      final_AUC_collection_test.append('none')
  else:
      final_BACC_collection_test.append(str(round(statistics.mean(BACC_collection_test),3))+'±'+str(round(statistics.stdev(BACC_collection_test),3)))
      final_Sn_collection_test.append(str(round(statistics.mean(Sn_collection_test),3))+'±'+str(round(statistics.stdev(Sn_collection_test),3)))
      final_Sp_collection_test.append(str(round(statistics.mean(Sp_collection_test),3))+'±'+str(round(statistics.stdev(Sp_collection_test),3)))
      final_MCC_collection_test.append(str(round(statistics.mean(MCC_collection_test),3))+'±'+str(round(statistics.stdev(MCC_collection_test),3)))
      final_AUC_collection_test.append(str(round(statistics.mean(AUC_collection_test),3))+'±'+str(round(statistics.stdev(AUC_collection_test),3)))
# performance in test
print(final_ACC_collection_test, '\n',final_BACC_collection_test, '\n',final_Sn_collection_test, '\n',final_Sp_collection_test, '\n',final_MCC_collection_test,'\n', final_AUC_collection_test)


Epoch 1/45
402/402 [==============================] - 3s 4ms/step - loss: 0.6674 - accuracy: 0.8254 - lr: 0.1000
Epoch 2/45
402/402 [==============================] - 2s 5ms/step - loss: 0.2244 - accuracy: 0.9116 - lr: 0.1000
Epoch 3/45
402/402 [==============================] - 2s 5ms/step - loss: 0.1701 - accuracy: 0.9299 - lr: 0.0600
Epoch 4/45
402/402 [==============================] - 2s 4ms/step - loss: 0.1523 - accuracy: 0.9401 - lr: 0.0600
Epoch 5/45
402/402 [==============================] - 2s 4ms/step - loss: 0.1420 - accuracy: 0.9436 - lr: 0.0600
Epoch 6/45
402/402 [==============================] - 2s 4ms/step - loss: 0.1232 - accuracy: 0.9504 - lr: 0.0360
Epoch 7/45
402/402 [==============================] - 2s 4ms/step - loss: 0.1154 - accuracy: 0.9534 - lr: 0.0360
Epoch 8/45
402/402 [==============================] - 2s 4ms/step - loss: 0.1078 - accuracy: 0.9580 - lr: 0.0360
Epoch 9/45
402/402 [==============================] - 2s 4ms/step - loss: 0.0938 - accuracy: 0.9

IndexError: ignored

In [ ]:
# performance in test
print(final_ACC_collection_test, '\n',final_BACC_collection_test, '\n',final_Sn_collection_test, '\n',final_Sp_collection_test, '\n',final_MCC_collection_test,'\n', final_AUC_collection_test)


['0.95±0.004', '0.938±0.018'] 
 ['0.95±0.004', '0.938±0.018'] 
 ['0.941±0.009', '0.929±0.021'] 
 ['0.958±0.007', '0.948±0.024'] 
 ['0.9±0.008', '0.877±0.036'] 
 ['0.989±0.002', '0.984±0.01']
